useful links:

- Data Preparation for Variable Length Input Sequences, URL: https://machinelearningmastery.com/data-preparation-variable-length-input-sequences-sequence-prediction/
- Masking and padding with Keras, URL: https://www.tensorflow.org/guide/keras/masking_and_padding
- Step-by-step understanding LSTM Autoencoder layers, URL: https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352XX, 
- Understanding input_shape parameter in LSTM with Keras, URL: https://stats.stackexchange.com/questions/274478/understanding-input-shape-parameter-in-lstm-with-keras
- tf.convert_to_tensor, URL: https://www.tensorflow.org/api_docs/python/tf/convert_to_tensor
- ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int) in Python, URL: https://datascience.stackexchange.com/questions/82440/valueerror-failed-to-convert-a-numpy-array-to-a-tensor-unsupported-object-type
- How to Identify and Diagnose GAN Failure Modes, URL: https://machinelearningmastery.com/practical-guide-to-gan-failure-modes/
- How to Develop a GAN for Generating MNIST Handwritten Digits
, URL: https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-an-mnist-handwritten-digits-from-scratch-in-keras/
- How to Visualize a Deep Learning Neural Network Model in Keras
, URL: https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/
- How to Implement GAN Hacks in Keras to Train Stable Models
, URL: https://machinelearningmastery.com/how-to-code-generative-adversarial-network-hacks/
- Tips for Training Stable Generative Adversarial Networks
, URL: https://machinelearningmastery.com/how-to-train-stable-generative-adversarial-networks/
- How to Implement GAN Hacks in Keras to Train Stable Models
, URL: https://machinelearningmastery.com/how-to-code-generative-adversarial-network-hacks/
- How to Configure Image Data Augmentation in Keras, URL: https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/


In [1]:
"""
* Copyright 2020, Maestria de Humanidades Digitales,
* Universidad de Los Andes
*
* Developed for the Msc graduation project in Digital Humanities
*
* This program is free software: you can redistribute it and/or modify
* it under the terms of the GNU General Public License as published by
* the Free Software Foundation, either version 3 of the License, or
* (at your option) any later version.
*
* This program is distributed in the hope that it will be useful,
* but WITHOUT ANY WARRANTY; without even the implied warranty of
* MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
* GNU General Public License for more details.
*
* You should have received a copy of the GNU General Public License
* along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""

# ===============================
# native python libraries
# ===============================
import re
import time
import random
import math
import json
import csv
import cv2
import datetime
import copy
import gc
from statistics import mean
from collections import OrderedDict
from collections import Counter
from collections import deque

# ===============================
# extension python libraries
# ===============================
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# natural language processing packages
import gensim
from gensim import models
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# downloading nlkt data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# sample handling sklearn package
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import make_scorer
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain

# # Keras + Tensorflow ML libraries
import tensorflow as tf
# from tensorflow.keras.layers

# preprocessing and processing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping

# models
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

# shapping layers
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate

# basic layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import TimeDistributed

# data processing layers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import SpatialDropout1D

# recurrent and convolutional layers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalMaxPool2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import ZeroPadding2D

# activarion function
from tensorflow.keras.layers import LeakyReLU

# optimization loss functions
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import SGD # OJO!
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam # OJO!
from tensorflow.keras.optimizers import Adadelta # OJO!
from tensorflow.keras.optimizers import Adagrad # OJO!

# image augmentation and processing
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

# ===============================
# developed python libraries
# ===============================

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# FUNCTION DEFINITION

In [2]:
'''
A UDF to convert input data into 3-D
array as required for LSTM network.

taken from https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352
'''
def temporalize(data, lookback):
    output_X = list()
    for i in range(len(X)-lookback-1):
        temp = list()
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            temp.append(data[[(i+j+1)], :])
        temp = np.array(temp, dtype="object")
        output_X.append(temp)
    output_X = np.array(output_X, dtype="object")
    return output_X

In [3]:
# function to read the image from file with cv2
def read_img(img_fpn):
    ans = cv2.imread(img_fpn, cv2.IMREAD_UNCHANGED)
    return ans

In [4]:
# fuction to scale the image and reduce cv2
def scale_img(img, scale_pct):

    width = int(img.shape[1]*scale_pct/100)
    height = int(img.shape[0]*scale_pct/100)
    dim = (width, height)
    # resize image
    ans = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return ans

In [5]:
# function to standarize image, has 2 types, from 0 to 1 and from -1 to 1
def std_img(img, minv, maxv, stype="std"):
    ans = None
    rangev = maxv - minv

    if stype == "std":
        ans = img.astype("float32")/float(rangev)
    
    elif stype == "ctr":
        rangev = float(rangev/2)
        ans = (img.astype("float32")-rangev)/rangev
    # ans = pd.Series(ans)
    return ans

In [6]:
# function to pad the image in the center
def pad_img(img, h, w, img_type):
    #  in case when you have odd number
    ans = None
    top_pad = np.floor((h - img.shape[0]) / 2).astype(np.uint8) # floor
    bottom_pad = np.ceil((h - img.shape[0]) / 2).astype(np.uint8)
    right_pad = np.ceil((w - img.shape[1]) / 2).astype(np.uint8)
    left_pad = np.floor((w - img.shape[1]) / 2).astype(np.uint8) # floor
    # print((top_pad, bottom_pad), (left_pad, right_pad))
    if img_type == "rgb":
        ans = np.copy(np.pad(img, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)), mode="constant", constant_values=0.0))   
    if img_type == "bw":
        ans = np.copy(np.pad(img, ((int(top_pad), int(bottom_pad)), (int(left_pad), int(right_pad))), mode="constant", constant_values=0))

    return ans

In [7]:
def update_shape(src_df, img_col, shape_col):

    ans = src_df
    src_col = list(ans[img_col])
    tgt_col = list()

    # ansdict = {}
    for data in src_col:
        tshape = data.shape
        tgt_col.append(tshape)

    ans[shape_col] = tgt_col
    return ans

In [8]:
# function to padd the images in the dataset, needs the shape, the type of image and the src + tgt columns of the frame to work with
def padding_images(src_df, src_col, tgt_col, max_shape, img_type):
    # ans = src_df
    src_images = src_df[src_col]
    tgt_images = list()
    max_x, max_y = max_shape[0], max_shape[1]

    for timg in src_images:        
        pimg = pad_img(timg, max_y, max_x, img_type)
        tgt_images.append(pimg)

    src_df[tgt_col] = tgt_images
    return src_df

In [9]:
# function to load the images in in memory
def get_images(rootf, src_df, src_col, tgt_col, scale_pct):
    ans = src_df
    src_files = list(ans[src_col])
    tgt_files = list()

    # ansdict = {}
    for tfile in src_files:
        tfpn = os.path.join(rootf, tfile)
        timg = read_img(tfpn)
        timg = scale_img(timg, scale_pct)
        tgt_files.append(timg)

    ans[tgt_col] = tgt_files
    return ans

In [10]:
# function to standarize the images in the dataset, it has 2 options
def standarize_images(src_df, src_col, tgt_col, img_type, std_opt):
    src_images = src_df[src_col]
    tgt_images = list()

    for timg in src_images:
        # pcolor image
        if img_type == "rgb":
            timg = np.asarray(timg, dtype="object")
        
        # b&w image
        if img_type == "rb":
            timg = np.asarray(timg) #, dtype="uint8")
            timg = timg[:,:,np.newaxis]
            timg = np.asarray(timg, dtype="object")
        
        # std_opt affect the standarization results
        # result 0.0 < std_timg < 1.0
        # result -1.0 < std_timg < 1.0
        std_timg = std_img(timg, 0, 255, std_opt)
        tgt_images.append(std_timg)

    src_df[tgt_col] = tgt_images
    return src_df

In [11]:
# function than rotates the original image to create a new example
def syth_rgb_img(data):

    samples = expand_dims(data, 0)
    datagen = ImageDataGenerator(rotation_range=90)
    ans = datagen.flow(samples, batch_size=1)
    ans = ans[0].astype("uint8")
    ans = np.squeeze(ans, 0)
    return ans

In [12]:
# function to get the max shape in the image dataset
def get_mshape(shape_data, imgt):

    max_x, max_y, max_ch = 0, 0, 0
    shape_data = list(shape_data)
    ans = None

    if imgt == "rgb":

        for tshape in shape_data:
            # tshape = eval(tshape)
            tx, ty, tch = tshape[0], tshape[1], tshape[2]

            if tx > max_x:
                max_x = tx
            if ty > max_y:
                max_y = ty
            if tch > max_ch:
                max_ch = tch
            
        ans = (max_x, max_y, max_ch)
    
    elif imgt == "bw":

        for tshape in shape_data:
            # tshape = eval(tshape)
            tx, ty = tshape[0], tshape[1]

            if tx > max_x:
                max_x = tx
            if ty > max_y:
                max_y = ty
            
        ans = (max_x, max_y)
        
    return ans

In [13]:
'''
A UDF to convert input data into 3-D
array as required for LSTM network.

taken from https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352
'''
def temporalize(data, lookback):
    output_X = list()
    for i in range(len(data)-lookback-1):
        temp = list()
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            temp.append(data[[(i+j+1)], :])
        temp = np.array(temp, dtype="object")
        output_X.append(temp)
    output_X = np.array(output_X, dtype="object")
    return output_X

In [14]:
# format the pandas df data into usable word dense vector representation, YOU NEED IT FOR THE CSV to be useful!
def format_dvector(work_corpus):

    ans = list()
    for dvector in work_corpus:
        dvector = eval(dvector)
        dvector = np.asarray(dvector)
        ans.append(dvector)
    ans = np.asarray(ans, dtype="object")
    return ans

In [15]:
# funct to concatenate all label columns into one for a single y in ML training, returns a list
def concat_labels(row, cname):

    ans = list()
    for c in cname:
        r = row[c]
        r = eval(r)
        ans = ans + r

    return ans

In [16]:
# function to save the ML model
def save_model(model, m_path, m_file):

    fpn = os.path.join(m_path, m_file)
    fpn = fpn + ".h5"
    # print(fpn)
    model.save(fpn)

In [17]:
# function to load the ML model
def load_model(m_path, m_file):

    fpn = os.path.join(m_path, m_file)
    fpn = fpn + ".h5"
    model = keras.models.load_model(fpn)
    return model

In [18]:
# function to cast dataframe and avoid problems with keras
def cast_batch(data):

    cast_data = list()

    if len(data) >= 2:

        for d in data:
            d = np.asarray(d).astype("float32")
            cast_data.append(d)

    return cast_data

In [19]:
# function to select real elements to train the discriminator
def gen_real_samples(data, sample_size, half_batch):

    real_data = list()
    rand_index = np.random.randint(0, sample_size, size=half_batch)

    # need at leas X, y
    # posible combinations are:
    # X_img/X_txt, y
    # X_img/X_txt, X_labels, y
    # X_img, X_txt, X_labels, y
    if len(data) >= 2:
        # selectinc the columns in the dataset
        for d in data:
            # td_real = d[rand_index]
            td_real = copy.deepcopy(d[rand_index])
            real_data.append(td_real)

    # casting data
    real_data = cast_batch(real_data)

    return real_data

In [20]:
# function to create fake elements to train the discriminator
def gen_fake_samples(gen_model, dataset_shape, half_batch):

    # fake data
    fake_data = None
    # conditional labels for the gan model
    conditional = dataset_shape.get("conditioned")
    # configuratin keys for the generator
    latent_dims = dataset_shape.get("latent_dims")
    cat_shape = dataset_shape.get("cat_shape")
    label_shape = dataset_shape.get("label_shape")
    data_cols = dataset_shape.get("data_cols")

    # generator config according to the dataset
    # X:images -> y:Real/Fake
    if data_cols == 2:
        # random textual latent space 
        latent_space = gen_latent_space(latent_dims, half_batch)
        # marking the images as fake in all accounts
        y_fake = gen_fake_negclass(cat_shape, half_batch)
        # random generated image from the model
        Xi_fake = gen_model.predict(latent_space)
        # fake samples
        fake_data = (Xi_fake, y_fake)

    # X_img, X_labels(classification), y (fake/real)
    elif (conditional == True) and data_cols == 3:
        # random textual latent space 
        latent_space = gen_latent_space(latent_dims, half_batch)
        # marking the images as fake in all accounts
        y_fake = gen_fake_negclass(cat_shape, half_batch)
        # marking all the images with fake labels
        Xl_fake = gen_fake_labels(label_shape, half_batch)

        # random generated image from the model
        Xi_fake = gen_model.predict([latent_space, Xl_fake])
        # fake samples
        fake_data = (Xi_fake, Xl_fake, y_fake)

    elif (conditional == False) and data_cols == 3:
        
        # random textual latent space 
        latent_space = gen_latent_space(latent_dims, half_batch)
        # marking the images as fake in all accounts
        y_fake = gen_fake_negclass(cat_shape, half_batch)
        # random generated image + text from the model
        Xi_fake, Xt_fake = gen_model.predict(latent_space)
        # fake samples
        fake_data = (Xi_fake, Xt_fake, y_fake)

    # X_img(rgb), X_txt(text), X_labels(classification), y (fake/real)
    elif data_cols == 4:

        # random textual latent space 
        latent_space = gen_latent_space(latent_dims, half_batch)
        # marking the images as fake in all accounts
        y_fake = gen_fake_negclass(cat_shape, half_batch)
        # marking all the images with fake labels
        Xl_fake = gen_fake_labels(label_shape, half_batch)

        # random generated image from the model
        Xi_fake, Xt_fake = gen_model.predict([latent_space, Xl_fake])
        # fake samples 
        fake_data = (Xi_fake, Xt_fake, Xl_fake, y_fake)

    # casting data type
    fake_data = cast_batch(fake_data)
    
    return fake_data

In [21]:
# function to create inputs to updalte the GAN generator
def gen_latent_data(dataset_shape, batch_size):

    # latent data
    latent_data = None

    # conditional labels for the gan model
    conditional = dataset_shape.get("conditioned")
    # configuratin keys for the generator
    latent_dims = dataset_shape.get("latent_dims")
    cat_shape = dataset_shape.get("cat_shape")
    label_shape = dataset_shape.get("label_shape")
    data_cols = dataset_shape.get("data_cols")

    # generator config according to the dataset
    # X:images -> y:Real/Fake
    if data_cols == 2:
        # random textual latent space 
        latent_space = gen_latent_space(latent_dims, batch_size)
        # marking the images as fake in all accounts
        y_gen = gen_fake_posclass(cat_shape, batch_size)
        # fake samples
        latent_data = (latent_space, y_gen)

    # X_img, X_labels(classification), y (fake/real)
    elif data_cols == 3 and (conditional == True):
        # random textual latent space 
        latent_space = gen_latent_space(latent_dims, batch_size)
        # marking the images as fake in all accounts
        y_gen = gen_fake_posclass(cat_shape, batch_size)
        # marking all the images with fake labels
        Xl_gen = gen_fake_labels(label_shape, batch_size)
        # gen samples
        latent_data = (latent_space, Xl_gen, y_gen)

    elif data_cols == 3 and (conditional == False):
        # random textual latent space 
        latent_space = gen_latent_space(latent_dims, batch_size)
        # marking the images as fake in all accounts
        y_gen = gen_fake_posclass(cat_shape, batch_size)
        # fake samples
        latent_data = (latent_space, y_gen)

    # X_img(rgb), X_txt(text), X_labels(classification), y (fake/real)
    elif data_cols == 4:
        # random textual latent space 
        latent_space = gen_latent_space(latent_dims, batch_size)
        # marking the images as fake in all accounts
        y_gen = gen_fake_posclass(cat_shape, batch_size)
        # marking all the images with fake labels
        Xl_gen = gen_fake_labels(label_shape, batch_size)
        # gen samples
        latent_data = (latent_space, Xl_gen, y_gen)

    return latent_data
# latent_gen = gen_latent_txt(latent_shape, batch_size)
# create inverted category for the fake noisy text
# y_gen = get_fake_positive(cat_shape[0], batch_size)

In [22]:
# function to generate random/latent text for the GAN generator
def gen_latent_space(latent_dims, n_samples):

    ans = None
    for i in range(n_samples):

        # noise = np.random.normal(0.0, 1.0, size=latent_shape)
        # noise = np.random.normal(0.0, 1.0, size=latent_dims)
        # noise = np.random.normal(0.5, 0.25, size=latent_shape)
        # noise = np.random.uniform(low=0.0, high=1.0, size=latent_shape)
        # noise = np.random.randn(latent_shape[0], latent_shape[1])
        noise = np.random.randn(latent_dims)
        if ans is None:
            txt = np.expand_dims(noise, axis=0)
            ans = txt
        else:
            txt = np.expand_dims(noise, axis=0)
            ans = np.concatenate((ans, txt), axis=0)
    return ans

In [23]:
# tfunction to smooth the fake positives
def smooth_positives(y):
	return y - 0.3 + (np.random.random(y.shape)*0.5)

In [24]:
# function to smooth the fake negatives
def smooth_negatives(y):
	return y + np.random.random(y.shape)*0.3

In [25]:
# function to smooth the data labels
def smooth_labels(y):
    # label smoothing formula
    # alpha: 0.0 -> original distribution, 1.0 uniform distribution
    # K: number of label classes
    # y_ls = (1 - alpha) * y_hot + alpha / K
    alpha = 0.3
    K = y[0].shape[0]
    ans = (1-alpha)*y + alpha/K
    return ans

In [26]:
# generate fake true categories for the generator
def gen_fake_posclass(cat_shape, batch_size):

    sz = (batch_size, cat_shape[0])
    ans = np.ones(sz)
    # smoothing fakes
    ans = smooth_positives(ans)
    ans = ans.astype("float32")
    return ans

In [27]:
# generate fake negative category to train the GAN
def gen_fake_negclass(cat_shape, batch_size):

    sz = (batch_size, cat_shape[0])
    ans = np.zeros(sz)
    ans = smooth_negatives(ans)
    ans = ans.astype("float32")
    return ans

In [28]:
# function to generate fake labels to train the GAN
def gen_fake_labels(label_shape, batch_size):

    sz = (batch_size, label_shape[0])
    ans = np.random.randint(0,1, size=sz)
    ans = smooth_labels(ans)
    ans = ans.astype("float32")
    return ans

In [29]:
# function to create text similar to the original one with 5% of noise
def syth_text(data, nptc=0.05):

    ans = None
    noise = np.random.normal(0, nptc, data.shape)
    ans = data + noise
    return ans

In [30]:
# synthetizing a noisy std image from real data
def syth_std_img(data):

    samples = np.expand_dims(data, 0)
    datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=10)
    # datagen = ImageDataGenerator(rotation_range=10, horizontal_flip=True, vertical_flip=True)
    ans = datagen.flow(samples, batch_size=1)
    ans = ans[0].astype("float32")
    ans = np.squeeze(ans, 0)
    return ans

In [31]:
# function to create new categories with some noise, default 5%
def syth_categories(data, nptc=0.05):

    ans = None
    noise = np.random.normal(0, nptc, data.shape)
    ans = data + noise
    return ans

In [32]:
# function to artificially span a batch with some noise and alterations by an specific number
# TODO fix because this is an old version with no flexibility
def expand_samples(data, synth_batch):

    X_txt = data[0]
    X_img = data[1]
    y = data[2]
    labels = data[3]

    # creating the exapnded batch response
    Xe_txt, Xe_img, ye, lbe = None, None, None, None

    # iterating in the original batch
    for Xtt, Xit, yt, lb in zip(X_txt, X_img, y, labels):

        # temporal synth minibatch per original image
        synth_Xt, synth_Xi, synth_y, synth_lb = None, None, None, None

        # synthetizing artificial data for the batch
        for i in range(synth_batch):

            # generating first element
            if (synth_Xt is None) and (synth_Xi is None) and (synth_y is None) and (synth_lb is None):
                # gen text
                gen_Xt = copy.deepcopy(Xtt)
                gen_Xt = np.array(gen_Xt)
                gen_Xt = np.expand_dims(gen_Xt, axis=0)
                synth_Xt = gen_Xt

                # gen images
                gen_Xi = syth_std_img(Xit)
                gen_Xi = np.expand_dims(gen_Xi, axis=0)
                synth_Xi = gen_Xi

                # gen category
                gen_yt = syth_categories(yt)
                gen_yt = np.expand_dims(gen_yt, axis=0)
                synth_y = gen_yt

                # gen labels
                gen_lb = syth_categories(lb)
                gen_lb = np.expand_dims(gen_lb, axis=0)
                synth_lb = gen_lb

            # generatin the rest of the elements
            else:
                # gen text
                gen_Xt = syth_text(Xtt)
                gen_Xt = np.expand_dims(gen_Xt, axis=0)
                synth_Xt = np.concatenate((synth_Xt, gen_Xt), axis=0)

                # gen images
                gen_Xi = syth_std_img(Xit)
                gen_Xi = np.expand_dims(gen_Xi, axis=0)
                synth_Xi = np.concatenate((synth_Xi, gen_Xi), axis=0)

                # gen category
                gen_yt = syth_categories(yt)
                gen_yt = np.expand_dims(gen_yt, axis=0)
                synth_y = np.concatenate((synth_y, gen_yt), axis=0)
        
                # gen labels
                gen_lb = syth_categories(lb)
                gen_lb = np.expand_dims(gen_lb, axis=0)
                synth_lb = np.concatenate((synth_lb, gen_lb), axis=0)

        # adding the first part to the training batch
        if (Xe_txt is None) and (Xe_img is None) and (ye is None) and (lbe is None):
            # adding text
            Xe_txt = synth_Xt
            # adding images
            Xe_img = synth_Xi
            # adding categories
            ye = synth_y
            # adding labels
            lbe = synth_lb

        # adding the rest of the batch
        else:
            # adding text
            Xe_txt = np.concatenate((Xe_txt, synth_Xt), axis=0)
            # adding images
            Xe_img = np.concatenate((Xe_img, synth_Xi), axis=0)
            # adding category
            ye = np.concatenate((ye, synth_y), axis=0)
            # adding labels
            lbe = np.concatenate((lbe, synth_lb), axis=0)

    Xe_txt, Xe_img, ye, lbe = cast_batch(Xe_txt, Xe_img, ye, lbe)

    e_data = (Xe_txt, Xe_img, ye, lbe)

    return e_data

In [33]:
# def drift_labels(Xt_real, Xi_real, y_real, Xt_fake, Xi_fake, y_fake, batch_size, drift_pct):
def drift_labels(real_data, fake_data, batch_size, drift_pct):

    # setting the size for the drift labels
    drift_size = int(math.ceil(drift_pct*batch_size))
    # random index for drift elements!!!
    rand_drifters = np.random.choice(batch_size, size=drift_size, replace=False)
    # print("batch size", batch_size, "\nrandom choise to change", drift_size, "\n", rand_drifters)

    # if the dataset has at leas X, y... NEED TO PASS A GOOD ORDER
    if (len(real_data) and len(fake_data)) >= 2:

        # iterating over the random choose index
        for drift in rand_drifters:

            # taking one real + fake column at a time
            # X_img/txt, y
            # X_img/txt, X_labels, y
            # X_img, X_txt, X_labels, y
            for real_col, fake_col in zip(real_data, fake_data):
                # copying real data in temporal var
                temp_drift = copy.deepcopy(real_col[drift])
                # replacing real with fakes
                real_col[drift] = copy.deepcopy(fake_col[drift])
                # updating fakes with temporal original
                fake_col[drift] = temp_drift

    return real_data, fake_data

In [34]:
# function to standarize image, has 2 types, from 0 to 1 and from -1 to 1
def inv_std_img(img, minv, maxv, stype="std"):
    ans = None
    rangev = maxv - minv

    if stype == "std":
        ans = img*rangev
        ans = np.asarray(ans).astype("uint8")

    elif stype == "ctr":
        rangev = float(rangev/2)
        ans = img+rangev
        ans = ans*rangev
        ans = np.asarray(ans).astype("uint8")

    return ans

In [35]:
# the function takes the ideas array, shape and configuration to render them into human understandable lenguage
# it select n number of ideas and plot them, for images, for text and for both
def plot_ideas(ideas, train_cfg, test_cfg):

    # get the index of random ideas in the set
    ideas_size = test_cfg.get("batch_size")
    gen_samples = test_cfg.get("gen_sample_size")
    data_cols = train_cfg.get("data_cols")

    # choosing non repeated ideas in the set
    rand_index = np.random.choice(ideas_size, size=gen_samples*gen_samples, replace=False)
    # print(rand_index)
    # print("ojo!!!!", len(ideas))
    # if the ideas are images or text
    if len(ideas) == 1:
        # print("data_cols:", data_cols)
        data = ideas[0]
        current_shape = data[0].shape
        # print("idea current_shape:", ideas.shape)
        # print("idea current_shape:", current_shape)

        if current_shape == train_cfg.get("img_shape"):
            render_painting(data, rand_index, train_cfg, test_cfg)

        elif current_shape == train_cfg.get("txt_shape"):
            render_wordcloud(data, rand_index, train_cfg, test_cfg)

    # if the ideas are images + text
    elif len(ideas) == 2:
        data_img = ideas[0]
        data_txt = ideas[1]
        render_painting(data_img, rand_index, train_cfg, test_cfg)
        render_wordcloud(data_txt, rand_index, train_cfg, test_cfg)

In [36]:
# this function takes the selected ideas and transform them into pytlot objects
def render_painting(ideas, rand_index, train_cfg, test_cfg):

    # get important data for iterating
    n_sample = test_cfg.get("gen_sample_size")
    report_fp_name = test_cfg.get("report_fn_path")
    epoch = test_cfg.get("current_epoch")

    # prep the figure
    fig, ax = plt.subplots(n_sample, n_sample, figsize=(20,20))
    fig.patch.set_facecolor("xkcd:white")

    # plot images
    for i in range(n_sample*n_sample):
        # define subplot
        plt.subplot(n_sample, n_sample, 1+i)

        # getting the images from sample
        rand_i = rand_index[i]
        gimg = ideas[rand_i]
        gimg = inv_std_img(gimg, 0, 255, "ctr")

        # turn off axis
        plt.axis("off")
        plt.imshow(gimg) #, interpolation="nearest")

    # plot leyend
    fig.suptitle("GENERATED PAINTINGS", fontsize=50)
    fig.legend()

    # save plot to file
    plot_name = "GAN-Gen-img-epoch%03d" % int(epoch)
    plot_name = plot_name + ".png"
    fpn = os.path.join(report_fp_name, "img", plot_name)
    plt.savefig(fpn)
    plt.close()

In [37]:
# this function takes the selected ideas and translate them into pytplot objects
def render_wordcloud(ideas, rand_index, train_cfg, test_cfg):

    # get important data for iterating
    n_sample = test_cfg.get("n_samples")
    lexicon = train_cfg.get("bow_lexicon")
    tfidf_tokens = train_cfg.get("tfidf_lexicon")
    # get important data for iterating
    n_sample = test_cfg.get("gen_sample_size")
    report_fp_name = test_cfg.get("report_fn_path")
    epoch = test_cfg.get("current_epoch")
    default = {"no":1, "words":1}

    # prep the figure
    fig, ax = plt.subplots(n_sample,n_sample, figsize=(20,20))
    fig.patch.set_facecolor("xkcd:white")

    # plot images
    for i in range(n_sample*n_sample):
        # define subplot
        plt.subplot(n_sample, n_sample, 1+i)
        
        # getting the images from sample
        rand_i = rand_index[i]
        gtxt = ideas[rand_i]
        gtxt = translate_from_lexicon(gtxt, tfidf_tokens, lexicon)

        wordcloud = WordCloud(max_font_size=100,
                                min_font_size=10,
                                max_words=100,
                                min_word_length=1,
                                relative_scaling = 0.5,
                                width=600, height=400,
                                background_color="white",
                                random_state=42)
        if len(gtxt) == 0:
            gtxt = default
        
        wordcloud.generate_from_frequencies(frequencies=gtxt)
        # plt.figure()

        # turn off axis
        plt.axis("off")
        plt.imshow(wordcloud, interpolation="bilinear") #, interpolation="nearest")

    # plot leyend
    fig.suptitle("GENERATED WORDS", fontsize=50)
    fig.legend()

    # save plot to file
    plot_name = "GAN-Gen-txt-epoch%03d" % int(epoch)
    plot_name = plot_name + ".png"
    fpn = os.path.join(report_fp_name, "txt", plot_name)
    plt.savefig(fpn)
    plt.close()

In [38]:
# this function loads the model known lexicon into the a dictionary for the world cloud to translate
def load_lexicon(lexicon_fp):

    lexicon = gensim.corpora.Dictionary.load(lexicon_fp)
    return lexicon

In [39]:
# this function takes the idtf dense word vector representacion and translate it to human lenguage using the kown lexicon
def translate_from_lexicon(tfidf_corpus, tfidf_dict, lexicon):

    wordcloud = dict()

    bow_corpus = tfidf2bow(tfidf_corpus, tfidf_dict)
    wordcloud = bow2words(bow_corpus, lexicon)
    return wordcloud

In [40]:
# translate from tfidf token representation to bow representation
def tfidf2bow(tfidf_corpus, tfidf_dict):

    bows = dict()
    tfidf_corpus = np.asarray(tfidf_corpus, dtype="float32")
    # print(type(tfidf_corpus))

    for tfidf_doc in tfidf_corpus:

        for tfidf_token in tfidf_doc:

            bows = get_similars(tfidf_token, bows, tfidf_dict)

    return bows

In [41]:
# stablish if the tfidf representation of a token is similar to the one in the tfidf dictionar
def get_similars(tfidf_token, bows, tfidf_dict):

    ans = bows
    ans = isclose_in(tfidf_token, bows, tfidf_dict)
    return ans

In [42]:
# this function return the similar values of the tfidf value with a token id and a count
def isclose_in(token, token_dict, cmp_tokens, tol=0.0001):

    for tcmp in cmp_tokens:
        for key, value in tcmp.items():

            if math.isclose(token, value, rel_tol=tol) and (key not in token_dict.keys()):
                token_dict.update({key:1})
            
            elif math.isclose(token, value, rel_tol=tol) and (key in token_dict.keys()):
                count = token_dict[key]
                count = count + 1
                token_dict.update({key:count})
    return token_dict

In [43]:
def bow2words(bow_txt, lexicon):

    words = dict()

    for key, value in bow_txt.items():
        token = lexicon.get(key)
        td = {token:value}
        # word = id2token.get(key)
        # td = {word:value}
        words.update(td)
    return words

In [44]:
# function to plot the generated images within a training epoch
def plot_gen_images(examples, epoch, report_fp_name, n_sample):

    # get important data for iterating
    example_size = examples.shape[0]
    og_shape = examples[0].shape
    rand_img = np.random.choice(example_size, size=n_sample*n_sample, replace=False) 
    # (0, example_size, size=n_sample*n_sample)

    # prep the figure
    fig, ax = plt.subplots(n_sample,n_sample, figsize=(20,20))
    fig.patch.set_facecolor("xkcd:white")

    # plot images
    for i in range(n_sample*n_sample):
        # define subplot
        plt.subplot(n_sample, n_sample, 1+i)

        # getting the images from sample
        rand_i = rand_img[i]
        gimg = examples[rand_i]
        gimg = inv_std_img(gimg, 0, 255, "ctr")

        # turn off axis
        plt.axis("off")
        plt.imshow(gimg) #, interpolation="nearest")

    # plot leyend
    fig.suptitle("GENERATED PAINTINGS", fontsize=50)
    fig.legend()

    # save plot to file
    plot_name = "GAN-Gen-img-epoch%03d" % int(epoch)
    plot_name = plot_name + ".png"
    fpn = os.path.join(report_fp_name, "img", plot_name)
    plt.savefig(fpn)
    plt.close()

In [45]:
# create a line plot of loss for the gan and save to file
def plot_metrics(disr_hist, disf_hist, gan_hist, report_fp_name, epoch):

    # reporting results
    disr_hist = np.array(disr_hist)
    disf_hist = np.array(disf_hist)
    gan_hist = np.array(gan_hist)

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,8))
    fig.patch.set_facecolor("xkcd:white")

    # loss
    ax1.plot(disr_hist[:,1], "royalblue", label="Loss: R-Dis")
    ax1.plot(disf_hist[:,1], "crimson", label="Loss: F-Dis")
    ax1.plot(gan_hist[:,1], "blueviolet", label="Loss: GAN/Gen")
    # ax1.plot(gan_hist[:], "blueviolet", label="Loss: GAN/Gen")

    # acc_
    ax2.plot(disr_hist[:,0], "royalblue", label="Acc: R-Dis")
    ax2.plot(disf_hist[:,0], "crimson", label="Acc: F-Dis")
    ax2.plot(gan_hist[:,0], "blueviolet", label="Acc: GAN/Gen")

    # plot leyend
    fig.suptitle("LEARNING BEHAVIOR", fontsize=20)
    ax1.grid(True)
    ax2.grid(True)
    ax1.set_title("Loss")
    ax2.set_title("Accuracy")
    ax1.set(xlabel = "Epoch [cycle]", ylabel = "Loss")
    ax2.set(xlabel = "Epoch [cycle]", ylabel = "Acc")
    fig.legend()

    # save plot to file
    plot_name = "GAN-learn-curve-epoch%03d" % int(epoch)
    plot_name = plot_name + ".png"
    fpn = os.path.join(report_fp_name, "learn", plot_name)
    plt.savefig(fpn)
    plt.close()

In [46]:
# function to calculate the loss and accuracy avg in multiple batchs of an epoch
def epoch_avg(log):
    loss, acc = None, None

    # if acc and loss are present to avg
    if type(log[0]) is list:
        if len(log) > 0:

            acc_list = list()
            loss_list = list()

            for l in log:
                ta = l[0]
                tl = l[1]

                acc_list.append(ta)
                loss_list.append(tl)

            loss, acc = mean(loss_list), mean(acc_list)
        return loss, acc
    
    else:
        # if only loss is present
        if len(log) > 0:

            loss_list = list()

            for l in log:
                loss_list.append(l)

            loss = mean(loss_list)
        return loss


In [47]:
# function to save model, needs the dirpath, the name and the datetime to save
def export_model(model, models_fp_name, filename, datetime):

    ss = True
    sln = True
    fext = "png"
    fpn = filename + "-" + datetime
    fpn = filename + "." + fext
    fpn = os.path.join(models_fp_name, fpn)
    plot_model(model, to_file=fpn, show_shapes=ss, show_layer_names=sln)

In [48]:
# function to format data to save in file
def format_metrics(disr_history, disf_history, gan_history):

    headers, data = None, None

    disr_hist = np.array(disr_history)
    disf_hist = np.array(disf_history)
    gan_hist = np.array(gan_history)

    # formating file headers
    headers = ["dis_loss_real", "dis_acc_real", "dis_loss_fake", "dis_acc_fake", "gen_gan_loss", "gen_gan_acc"]
    # headers = ["dis_loss_real", "dis_acc_real", "dis_loss_fake", "dis_acc_fake", "gen_gan_loss",] # "gen_gan_acc"]

    # formating fake discriminator train data
    drhl = disr_hist[:,1]
    drha = disr_hist[:,0]

    # formating real discrimintator train data
    dfhl = disf_hist[:,1]
    dfha = disf_hist[:,0]

    # formating gan/gen train data
    # gghl = gan_hist[:]# .flatten()
    gghl = gan_hist[:,1]
    ggha = gan_hist[:,0]

    # adding all formatted data into list
    data = np.column_stack((drhl, drha, dfhl, dfha, gghl, ggha))
    # data = np.column_stack((drhl, drha, dfhl, dfha, gghl)) #, ggha))

    return data, headers

In [49]:
# function to write data in csv file
def write_metrics(data, headers, report_fn_path, filename):

    # print(report_fn_path, filename)
    fpn = filename + "-train-history.csv"
    fpn = os.path.join(report_fn_path, fpn)

    history_df = pd.DataFrame(data, columns=headers)
    tdata = history_df.to_csv(
                            fpn,
                            sep=",",
                            index=False,
                            encoding="utf-8",
                            mode="w",
                            quoting=csv.QUOTE_ALL
                            )

In [50]:
# function to safe the loss/acc logs in training for the gan/gen/dis models
def save_metrics(disr_history, disf_history, gan_history, report_fn_path, filename):

    data, headers = format_metrics(disr_history, disf_history, gan_history)
    write_metrics(data, headers, report_fn_path, filename)

In [51]:
# function to know the time between epochs or batchs it return the new time for a new calculation
def lapse_time(last_time, epoch):

    now_time = datetime.datetime.now()
    deltatime = now_time - last_time
    deltatime = deltatime.total_seconds()
    deltatime = "%.2f" % deltatime
    msg = "Epoch:%3d " % int(epoch+1)
    msg = msg + "elapsed time: " + str(deltatime) + " [s]"
    print(msg)
    return now_time

In [52]:
# function to test the model while training
def test_model(gen_model, dis_model, data, data_shape, train_cfg, test_cfg): 

    dataset_size = test_cfg.get("dataset_size")
    batch_size = test_cfg.get("batch_size")
    synth_batch = test_cfg.get("synth_batch")
    epoch = int(test_cfg.get("current_epoch"))
    report_fn_path = test_cfg.get("report_fn_path")
    gen_samples = test_cfg.get("gen_sample_size") 
    balance_batch = test_cfg.get("balance_batch")
    split_batch = int(batch_size/2)

    # select real txt2img for discrimintator
    real_data = gen_real_samples(data, dataset_size, batch_size)

    # create false txt for txt2img for generator
    fake_data = gen_fake_samples(gen_model, data_shape, batch_size)

    # expand the training sample for the discriminator
    if synth_batch > 1:
        real_data = expand_samples(real_data, synth_batch)
        fake_data = expand_samples(fake_data, synth_batch)

    # balance training samples for the discriminator
    if balance_batch == True:
        real_data = balance_samples(real_data)
        fake_data = balance_samples(fake_data)
        
    # print(Xt_real.shape, Xi_real.shape, y_real.shape, yl_real.shape)
    # print(Xt_fake.shape, Xi_fake.shape, y_fake.shape, yl_fake.shape)

    # plotting gen ideas
    # ideas = (fake_data[0], fake_data[1])
    # plot_ideas(ideas, train_cfg, test_cfg)

    # test metrics
    test_real, test_fake = None, None

    # 1 output, img or txt
    if len(data) == 2:
        ideas = (fake_data[0],)
        print(len(ideas))
        plot_ideas(ideas, train_cfg, test_cfg)
        test_real, test_fake = test_gan(dis_model, real_data, fake_data, batch_size)

    # 2 output, img + txt and labels conditioned
    elif len(data) == 3 and data_shape.get("conditioned") == True:
        ideas = (fake_data[0],)
        plot_ideas(ideas, train_cfg, test_cfg)
        test_real, test_fake = test_cgan(dis_model, real_data, fake_data, batch_size)

    # 2 output, img + txt unconditioned
    elif len(data) == 3 and data_shape.get("conditioned") == False:
        ideas = (fake_data[0], fake_data[1])
        plot_ideas(ideas, train_cfg, test_cfg)
        test_real, test_fake = test_multi_gan(dis_model, real_data, fake_data, batch_size)

    # 2 outputs, img + txt and label conditioned
    elif len(data) == 4:
        ideas = (fake_data[0], fake_data[1])
        plot_ideas(ideas, train_cfg, test_cfg)
        test_real, test_fake = test_multi_cgan(dis_model, real_data, fake_data, batch_size)

    # summarize discriminator performance
    print("Batch Size %d -> Samples: Fake: %d & Real: %d" % (batch_size*synth_batch, split_batch, split_batch))
    print(">>> Test Fake -> Acc: %.3f || Loss: %.3f" % (test_fake[1], test_fake[0]))
    print(">>> Test Real -> Acc: %.3f || Loss: %.3f" % (test_real[1], test_real[0]))
    # print(">>> Test Gen -> Acc: %.3f || Loss: %.3f" % (test_cgen[1], test_cgen[0]))

In [53]:
# special function to train the GAN
# https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-an-mnist-handwritten-digits-from-scratch-in-keras/
# def train(gen_model, dis_model, gan_model, X_img, X_txt, y, labels, epochs, batch_size, save_intervas, fn_config):
def training_model(gen_model, dis_model, gan_model, data, train_cfg): # epochs, batch_size, save_intervas, fn_config

    # sample size
    dataset_size = train_cfg.get("dataset_size")

    # data shape for the generator
    data_shape = {
        "latent_dims": train_cfg.get("latent_dims"),
        "cat_shape": train_cfg.get("cat_shape"),
        "txt_shape": train_cfg.get("txt_shape"),
        "label_shape": train_cfg.get("label_shape"),
        "conditioned": train_cfg.get("conditioned"),
        "data_cols": train_cfg.get("data_cols"),
        }
    # print(data_shape)

    # augmentation factor
    synth_batch = train_cfg.get("synth_batch")
    balance_batch = train_cfg.get("balance_batch")
    n = train_cfg.get("gen_sample_size")

    epochs = train_cfg.get("max_epochs")
    batch_size = train_cfg.get("batch_size")
    half_batch = int(batch_size/2)
    batch_per_epoch = int(dataset_size/batch_size)
    # fake/real batch division
    real_batch = int((batch_size*synth_batch)/2)

    # train config
    model_fn_path = train_cfg.get("models_fn_path")
    report_fn_path = train_cfg.get("report_fn_path")
    dis_model_name = train_cfg.get("dis_model_name")
    gen_model_name = train_cfg.get("gen_model_name")
    gan_model_name = train_cfg.get("gan_model_name")
    check_intervas = train_cfg.get("check_epochs")
    save_intervas = train_cfg.get("save_epochs")
    max_models = train_cfg.get("max_models")
    pretrain = train_cfg.get("pretrained")

	# prepare lists for storing stats each epoch
    disf_hist, disr_hist, gan_hist = list(), list(), list()
    train_time = None

    # train dict config
    test_cfg = {
        "report_fn_path": report_fn_path,
        "dataset_size": dataset_size,
        "batch_size": batch_size,
        "synth_batch": synth_batch,
        "gen_sample_size": train_cfg.get("gen_sample_size"),
        "current_epoch": None,
    }

    # iterating in training epochs:
    for ep in range(epochs+1):
        # epoch logs
        ep_disf_hist, ep_disr_hist, ep_gan_hist = list(), list(), list()
        train_time = datetime.datetime.now()

        # iterating over training batchs
        for batch in range(batch_per_epoch):

            # select real txt2img for discrimintator
            real_data = gen_real_samples(data, dataset_size, half_batch)
            # create false txt for txt2img for generator
            fake_data = gen_fake_samples(gen_model, data_shape, half_batch)

            # expand the training sample for the discriminator
            if synth_batch > 1:
                real_data = expand_samples(real_data, synth_batch)
                fake_data = expand_samples(fake_data, synth_batch)

            # balance training samples for the discriminator
            if balance_batch == True:
                real_data = balance_samples(real_data)
                fake_data = balance_samples(fake_data)

            # print(Xt_real.shape, Xi_real.shape, y_real.shape, yl_real.shape)
            # print(Xt_fake.shape, Xi_fake.shape, y_fake.shape, yl_fake.shape)
            # print(real_data[0].shape, fake_data[0].shape)
            # print(real_data[1].shape, fake_data[1].shape)
            # drift labels to confuse the model
            real_data, fake_data = drift_labels(real_data, fake_data, half_batch, 0.05)

            # TODO transfor this in 1 function train_model()...
            dhf, dhr, gh = None, None, None

            if len(data) == 2:
                dhr, dhf, gh = train_gan(dis_model, gan_model, real_data, fake_data, batch_size, data_shape)

            elif len(data) == 3 and data_shape.get("conditioned") == True:
                dhr, dhf, gh = train_cgan(dis_model, gan_model, real_data, fake_data, batch_size, data_shape)

            elif len(data) == 3 and data_shape.get("conditioned") == False:
                # TODO need to implement this function!!!
                dhr, dhf, gh = train_multi_gan(dis_model, gan_model, real_data, fake_data, batch_size, data_shape)

            elif len(data) == 4:
                dhr, dhf, gh = train_multi_cgan(dis_model, gan_model, real_data, fake_data, batch_size, data_shape)

            # epoch log
            ep_disr_hist.append(dhr)
            ep_disf_hist.append(dhf)
            ep_gan_hist.append(gh)

			# print('>%d, %d/%d, dis_=%.3f, gen=%.3f' % (ep+1, batch+1, bat_per_epo, dis_history, gen_history))
            log_msg = ">>> Epoch: %d, B/Ep: %d/%d, Batch S: %d" %(ep+1, batch+1, batch_per_epoch, batch_size*synth_batch)
            log_msg = "%s -> [R-Dis loss: %.3f, acc: %.3f]" % (log_msg, dhr[0], dhr[1])
            log_msg = "%s || [F-Dis loss: %.3f, acc: %.3f]" % (log_msg, dhf[0], dhf[1])
            log_msg = "%s || [Gen loss: %.3f, acc: %.3f]" % (log_msg, gh[0], gh[1])
            print(log_msg)

        # record history for epoch
        disr_hist.append(epoch_avg(ep_disr_hist))
        disf_hist.append(epoch_avg(ep_disf_hist))
        gan_hist.append(epoch_avg(ep_gan_hist))
        test_cfg["current_epoch"] = ep
        
		# evaluate the model performance sometimes
        if (ep) % check_intervas == 0:
            print("Epoch:", ep+1, "Testing model training process...")
            
            # test_model(gen_model, dis_model, data, data_shape, test_cfg) #, synth_batch)
            test_model(gen_model, dis_model, data, data_shape, train_cfg, test_cfg)
            print("Ploting results")
            plot_metrics(disr_hist, disf_hist, gan_hist, report_fn_path, ep)
            save_metrics(disr_hist, disf_hist, gan_hist, report_fn_path, gan_model_name)

		# saving the model sometimes
        if (ep) % save_intervas == 0:
            print("Epoch:", ep+1, "Saving the training progress...")
            save_models(dis_model, gen_model, gan_model, train_cfg, test_cfg)
            clear_models(train_cfg, test_cfg)
        
        train_time = lapse_time(train_time, ep)

In [54]:
def save_models(dis_model, gen_model, gan_model, train_cfg, test_cfg):

    ep = test_cfg.get("current_epoch") # = ep
    epoch_sufix = "-epoch%d" % int(ep)

    model_fn_path = train_cfg.get("models_fn_path")
    dis_model_name = train_cfg.get("dis_model_name")
    gen_model_name = train_cfg.get("gen_model_name")
    gan_model_name = train_cfg.get("gan_model_name")

    # epoch_sufix = "-last"
    epoch_sufix = str(epoch_sufix)
    dis_mn = dis_model_name + epoch_sufix
    gen_mn = gen_model_name + epoch_sufix
    gan_mn = gan_model_name + epoch_sufix

    dis_path = os.path.join(model_fn_path, "Dis")
    gen_path = os.path.join(model_fn_path, "Gen")
    gan_path = os.path.join(model_fn_path, "GAN")

    save_model(dis_model, dis_path, dis_mn)
    save_model(gen_model, gen_path, gen_mn)
    save_model(gan_model, gan_path, gan_mn)


In [55]:
def clear_models(train_cfg, test_cfg):

    # epoch_sufix = "-epoch%d" % int(ep)
    model_fn_path = train_cfg.get("models_fn_path")
    dis_path = os.path.join(model_fn_path, "Dis")
    gen_path = os.path.join(model_fn_path, "Gen")
    gan_path = os.path.join(model_fn_path, "GAN")
    max_files = train_cfg.get("max_save_models")

    list_path = (dis_path, gen_path, gan_path)
    rmv_path = list()

    for path in list_path:

        files = os.listdir(path)
        filepaths = list()

        for f in files:
            fp = os.path.join(path, f)
            filepaths.append(fp)

        # print(filepaths)
        filepaths.sort(key=os.path.getctime, reverse=True)
        # print("files!!!!", filepaths)

        if len(filepaths) > max_files:

            for del_file in filepaths[max_files:]:
                os.remove(del_file)


In [56]:
def train_gan(dis_model, gan_model, real_data, fake_data, batch_size, dataset_shape):

    # real data asignation
    Xi_real = real_data[0]
    y_real = real_data[1]

    # fake data asignation
    Xi_fake = fake_data[0]
    y_fake = fake_data[1]

    # train for real samples batch
    dhr = dis_model.train_on_batch(Xi_real, y_real)
    # train for fake samples batch
    dhf = dis_model.train_on_batch(Xi_fake, y_fake)

    # prepare text and inverted categories from the latent space as input for the generator
    latent_gen, y_gen = gen_latent_data(dataset_shape, batch_size)

    # update the generator via the discriminator's error
    gh = gan_model.train_on_batch(latent_gen, y_gen)

    return dhr, dhf, gh

In [57]:
def train_cgan(dis_model, gan_model, real_data, fake_data, batch_size, dataset_shape):

    # real data asignation
    Xi_real = real_data[0]
    yl_real = real_data[1]
    y_real = real_data[2]

    # fake data asignation
    Xi_fake = fake_data[0]
    yl_fake = fake_data[1]
    y_fake = fake_data[2]

    # train for real samples batch
    dhr = dis_model.train_on_batch([Xi_real, yl_real], y_real)
    # train for fake samples batch
    dhf = dis_model.train_on_batch([Xi_fake, yl_fake], y_fake)

    # prepare text and inverted categories from the latent space as input for the generator
    latent_gen, yl_gen, y_gen = gen_latent_data(dataset_shape, batch_size)

    # update the generator via the discriminator's error
    gh = gan_model.train_on_batch([latent_gen, yl_gen], y_gen)

    return dhr, dhf, gh

In [58]:
def train_multi_cgan(dis_model, gan_model, real_data, fake_data, batch_size, dataset_shape):

    # real data asignation
    Xi_real = real_data[0]
    Xt_real = real_data[1]
    Xl_real = real_data[2]
    y_real = real_data[3]

    # fake data asignation
    Xi_fake = fake_data[0]
    Xt_fake = fake_data[1]
    Xl_fake = fake_data[2]
    y_fake = fake_data[3]

    # train for real samples batch
    dhr = dis_model.train_on_batch([Xi_real, Xt_real, Xl_real], y_real)
    # train for fake samples batch
    dhf = dis_model.train_on_batch([Xi_fake, Xt_fake, Xl_fake], y_fake)

    # prepare text and inverted categories from the latent space as input for the generator
    latent_gen, yl_gen, y_gen = gen_latent_data(dataset_shape, batch_size)

    # update the generator via the discriminator's error
    gh = gan_model.train_on_batch([latent_gen, yl_gen], y_gen)

    return dhr, dhf, gh

In [59]:
def test_gan(dis_model, real_data, fake_data, batch_size):
    
    # drift labels to confuse the model
    real_data, fake_data = drift_labels(real_data, fake_data, batch_size, 0.05)

    # real data asignation
    Xi_real = real_data[0]
    y_real = real_data[1]

    # fake data asignation
    Xi_fake = fake_data[0]
    y_fake = fake_data[1]

    # evaluate model
    test_real = dis_model.evaluate(Xi_real, y_real, verbose=0)
    test_fake = dis_model.evaluate(Xi_fake, y_fake, verbose=0)

    return test_real, test_fake

In [60]:
def test_cgan(dis_model, real_data, fake_data, batch_size):
    
    # drift labels to confuse the model
    real_data, fake_data = drift_labels(real_data, fake_data, batch_size, 0.05)

    # real data asignation
    Xi_real = real_data[0]
    Xl_real = real_data[1]
    y_real = real_data[2]

    # fake data asignation
    Xi_fake = fake_data[0]
    Xl_fake = fake_data[1]
    y_fake = fake_data[2]

    # evaluate model
    test_real = dis_model.evaluate([Xi_real, Xl_real], y_real, verbose=0)
    test_fake = dis_model.evaluate([Xi_fake, Xl_fake], y_fake, verbose=0)

    return test_real, test_fake

In [61]:
def test_multi_cgan(dis_model, real_data, fake_data, batch_size):

    # drift labels to confuse the model
    real_data, fake_data = drift_labels(real_data, fake_data, batch_size, 0.05)

    # real data asignation
    Xi_real = real_data[0]
    Xt_real = real_data[1]
    Xl_real = real_data[2]
    y_real = real_data[3]

    # fake data asignation
    Xi_fake = fake_data[0]
    Xt_fake = fake_data[1]
    Xl_fake = fake_data[2]
    y_fake = fake_data[3]

    # evaluate model
    test_real = dis_model.evaluate([Xi_real, Xt_real, Xl_real], y_real, verbose=0)
    test_fake = dis_model.evaluate([Xi_fake, Xt_fake, Xl_fake], y_fake, verbose=0)

    return test_real, test_fake

# EXEC SCRIPT

## Dataset prep

In [62]:
# variable definitions
# root folder
dataf = "Data"

# subfolder with predictions txt data
imagef = "Img"

# report subfolder
reportf = "Reports"

#  subfolder with the CSV files containing the ML pandas dataframe
trainf = "Train"
testf = "Test"

# subfolder for model IO
modelf = "Models"

# dataframe file extension
fext = "csv"
imgf = "jpg"
lexf = "dict"

rgb_sufix = "rgb"
bw_sufix = "bw"

# standard sufix
stdprefix = "std-"

# ml model useful data
mltprefix = "ml-"

# report names
# timestamp = datetime.date.today().strftime("%d-%b-%Y")
timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

sample_sufix = "Small"
# sample_sufix = "Large"
# sample_sufix = "Paintings"
imgf_sufix = "Img-Data-"
text_sufix = "Text-Data-"

# std-VVG-Gallery-Text-Data-Paintings
gallery_prefix = "VVG-Gallery-"

# dataframe file name
text_fn = stdprefix + gallery_prefix + text_sufix + sample_sufix + "." + fext
imgf_fn = stdprefix + gallery_prefix + imgf_sufix + sample_sufix + "." + fext
valt_fn = "Validation-GAN-" + text_sufix + sample_sufix + "." + fext
lexicon_fn = gallery_prefix + text_sufix + sample_sufix + "." + lexf

# model names
dis_model_name = "VVG-Text2Img-CDiscriminator"
gen_model_name = "VVG-Text2Img-CGenerator"
gan_model_name = "VVG-Text2Img-CGAN"

# to continue training after stoping script
continue_training = True

# ramdom seed
randseed = 42

# sample distribution train vs test sample size
train_split = 0.80
test_split = 1.0 - train_split

# regex to know that column Im interested in
keeper_regex = r"(^ID$)|(^std_)"

imgt = rgb_sufix
# imgt = bw_sufix

# woring values for code
work_txtf, work_imgf, work_sufix, work_imgt, work_lex = text_fn, imgf_fn, sample_sufix, imgt, lexicon_fn

print("=== working files ===")
print("\n", work_txtf, "\n", work_imgf, "\n", work_sufix, "\n", work_imgt, "\n", valt_fn, "\n", work_lex)

=== working files ===

 std-VVG-Gallery-Text-Data-Small.csv 
 std-VVG-Gallery-Img-Data-Small.csv 
 Small 
 rgb 
 Validation-GAN-Text-Data-Small.csv 
 VVG-Gallery-Text-Data-Small.dict


In [63]:
root_folder = os.getcwd()
root_folder = os.path.split(root_folder)[0]
root_folder = os.path.normpath(root_folder)
print(root_folder)

c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer


In [64]:
# variable reading
# dataframe filepath for texttual data
text_fn_path = os.path.join(root_folder, dataf, trainf, work_txtf)
print(text_fn_path, os.path.exists(text_fn_path))

# dataframe filepath for img data
img_fn_path = os.path.join(root_folder, dataf, trainf, work_imgf)
print(img_fn_path, os.path.exists(img_fn_path))

# dictionary filepath for the GAN data
lex_fn_path = os.path.join(root_folder, dataf, trainf, work_lex)
print(lex_fn_path, os.path.exists(lex_fn_path))

# dataframe filepath for GAN data
val_fn_path = os.path.join(root_folder, dataf, testf, valt_fn)
print(val_fn_path, os.path.exists(val_fn_path))

# filepath for the models
model_fn_path = os.path.join(root_folder, dataf, modelf)
print(model_fn_path, os.path.exists(model_fn_path))

# filepath for the reports
report_fn_path = os.path.join(root_folder, dataf, reportf)
print(report_fn_path, os.path.exists(report_fn_path))

c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Train\std-VVG-Gallery-Text-Data-Small.csv True
c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Train\std-VVG-Gallery-Img-Data-Small.csv True
c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Train\VVG-Gallery-Text-Data-Small.dict True
c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Test\Validation-GAN-Text-Data-Small.csv False
c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Models True
c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Reports True


In [65]:
# rading training data
# loading textual file
text_df = pd.read_csv(
                text_fn_path,
                sep=",",
                encoding="utf-8",
                engine="python",
            )
text_cols = text_df.columns.values

# loading image file
img_df = pd.read_csv(
                img_fn_path,
                sep=",",
                encoding="utf-8",
                engine="python",
            )
img_cols = img_df.columns.values

In [66]:
idx_cols = list()

for tcol in text_cols:
    if tcol in img_cols:
        idx_cols.append(tcol)
print(idx_cols)

source_df = pd.merge(text_df, img_df, how="inner", on=idx_cols)

['ID', 'F-number', 'JH-number', 'creator-date', 'creator-place', 'Dimensions', 'details', 'std_cat_creator-date', 'std_cat_creator-place', 'std_cat_Dimensions', 'std_cat_details']


In [67]:
# checking everything is allrigth
img_df = None
text_df = None
source_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 58
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     59 non-null     object
 1   F-number               59 non-null     object
 2   JH-number              59 non-null     object
 3   creator-date           59 non-null     object
 4   creator-place          59 non-null     object
 5   Dimensions             59 non-null     object
 6   details                59 non-null     object
 7   MUS_TEXT               59 non-null     object
 8   std_cat_creator-date   59 non-null     object
 9   std_cat_creator-place  59 non-null     object
 10  std_cat_Dimensions     59 non-null     object
 11  std_cat_details        59 non-null     object
 12  clr_tokens             59 non-null     object
 13  lemmas                 59 non-null     object
 14  bows_tokens            59 non-null     object
 15  idxs_tokens            59

In [68]:
source_df = source_df.set_index("ID")

In [69]:
# reading images from folder and loading images into df
# working variables
src_col = work_imgt + "_img"
tgt_col = work_imgt + "_img" + "_data"
work_shape = work_imgt + "_shape"
scale = 32 # !!! 50->400pix, 64->512pix, 32->256pix
print(src_col, tgt_col)
source_df = get_images(root_folder, source_df, src_col, tgt_col, scale)

rgb_img rgb_img_data


In [70]:
# update image shape
source_df = update_shape(source_df, tgt_col, work_shape)

In [71]:
# searching the biggest shape in the image files
print(work_shape)
shape_data = source_df[work_shape]
max_shape = get_mshape(shape_data, work_imgt)
print(max_shape)

rgb_shape
(256, 256, 3)


In [72]:
# padding training data according to max shape of the images in gallery
pad_prefix = "pad_"
conv_prefix = "cnn_"
src_col = work_imgt + "_img" + "_data"
tgt_col = pad_prefix + conv_prefix + src_col

print(src_col, tgt_col, work_imgt)
source_df = padding_images(source_df, src_col, tgt_col, max_shape, work_imgt)

rgb_img_data pad_cnn_rgb_img_data rgb


In [73]:
# reading images from folder and stadarizing images into df
# working variables
print("standarizing regular images...")
src_col = work_imgt + "_img" + "_data"
tgt_col = "std_" + src_col

# source_df = standarize_images(source_df, src_col, tgt_col)

standarizing regular images...


In [74]:
print("standarizing padded images...")
src_col = pad_prefix + conv_prefix + work_imgt + "_img" + "_data"
tgt_col = "std_" + src_col
print(src_col, tgt_col)

# std_opt = "std"
std_opt = "ctr"
source_df = standarize_images(source_df, src_col, tgt_col, work_imgt, std_opt)

standarizing padded images...
pad_cnn_rgb_img_data std_pad_cnn_rgb_img_data


In [75]:
# shuffle the DataFrame rows
source_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, s0004V1962r to d1125S2005
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   F-number                  59 non-null     object
 1   JH-number                 59 non-null     object
 2   creator-date              59 non-null     object
 3   creator-place             59 non-null     object
 4   Dimensions                59 non-null     object
 5   details                   59 non-null     object
 6   MUS_TEXT                  59 non-null     object
 7   std_cat_creator-date      59 non-null     object
 8   std_cat_creator-place     59 non-null     object
 9   std_cat_Dimensions        59 non-null     object
 10  std_cat_details           59 non-null     object
 11  clr_tokens                59 non-null     object
 12  lemmas                    59 non-null     object
 13  bows_tokens               59 non-null     object
 14  idxs_tokens    

In [76]:
# cleaning memory
gc.collect()

62

In [77]:
# function to find a name of column names according to a regex
def get_keeper_cols(col_names, search_regex):
    ans = [i for i in col_names if re.search(search_regex, i)]
    return ans

In [78]:
# function to find the disperse columns in the df
def get_disperse_categories(src_df, keep_cols, max_dis, check_cols, ignore_col):

    ans = list()

    max_dis = 2
    tcount = 0

    while tcount < max_dis:
        for label_col in keep_columns:

            if label_col != ignore_col:

                label_count = src_df[label_col].value_counts(normalize=False)

                if tcount < label_count.shape[0] and (check_cols in label_col):
                    tcount = label_count.shape[0]
                    ans.append(label_col)
                # print("count values of", label_col, ":=", label_count.shape)#.__dict__)
        tcount = tcount + 1
    
    return ans

In [79]:
# function to remove the disperse columns from the interesting ones
def remove_disperse_categories(keep_columns, too_disperse):
    for too in too_disperse:
        keep_columns.remove(too)
    return keep_columns

In [80]:
def padding_corpus(train_df, dvector_col, pad_prefix):
    # getting the corpus dense vectors
    work_corpus = np.asarray(train_df[dvector_col], dtype="object")

    # converting list of list to array of array
    print("Original txt shape", work_corpus.shape)

    # padding the representation
    work_corpus = pad_sequences(work_corpus, dtype='object', padding="post")
    # print("Padded txt shape", work_corpus.shape)

    # creating the new column and saving padded data
    padded_col_dvector = pad_prefix + dvector_col

    # print(padded_col)
    train_df[padded_col_dvector] = list(work_corpus)
    print("Padded txt shape", work_corpus.shape)
    return train_df

In [81]:
def heat_categories(train_df, cat_cols, tgt_col):

    labels_data = train_df[cat_cols]
    labels_concat = list()

    # concatenating all category labels from dataframe
    for index, row in labels_data.iterrows():
        row = concat_labels(row, labels_cols)
        labels_concat.append(row)

    # print(len(labels_concat[0]), type(labels_concat[0]))
    # updating dataframe
    tcat_label_col = "std_cat_labels"
    train_df[tgt_col] = labels_concat

    return train_df

In [82]:
# function to adjust the textual data for the LSTM layers in the model
def format_corpus(corpus, timesteps, features):

    # preparation for reshape lstm model
    corpus = temporalize(corpus, timesteps)
    print(corpus.shape)

    corpus = corpus.reshape((corpus.shape[0], timesteps, features))
    print(corpus.shape)

    return corpus

In [83]:
# selecting data to train
# want to keep the columns starting with STD_
keep_columns = list(source_df.columns)
print("------ original input/interested columns ------")
print(keep_columns)

# create the columns Im interesting in
keep_columns = get_keeper_cols(keep_columns, keeper_regex)
# keep_columns = [i for i in df_columns if re.search(keeper_regex, i)]

print("\n\n------ Interesting columns ------")
print(keep_columns)

------ original input/interested columns ------
['F-number', 'JH-number', 'creator-date', 'creator-place', 'Dimensions', 'details', 'MUS_TEXT', 'std_cat_creator-date', 'std_cat_creator-place', 'std_cat_Dimensions', 'std_cat_details', 'clr_tokens', 'lemmas', 'bows_tokens', 'idxs_tokens', 'tfidf_tokens', 'std_dvec_tokens', 'rgb_img', 'bw_img', 'rgb_shape', 'bw_shape', 'rgb_img_data', 'pad_cnn_rgb_img_data', 'std_pad_cnn_rgb_img_data']


------ Interesting columns ------
['std_cat_creator-date', 'std_cat_creator-place', 'std_cat_Dimensions', 'std_cat_details', 'std_dvec_tokens', 'std_pad_cnn_rgb_img_data']


In [84]:
too_disperse = get_disperse_categories(source_df, keep_columns, 2, "std_cat_", "std_pad_cnn_rgb_img_data")
print(too_disperse)

['std_cat_creator-date', 'std_cat_Dimensions']


In [85]:
# creating the training dataframe
keep_columns = remove_disperse_categories(keep_columns, too_disperse)
# keep_columns.remove("ID")
print("------ Interesting columns ------")
print(keep_columns)

------ Interesting columns ------
['std_cat_creator-place', 'std_cat_details', 'std_dvec_tokens', 'std_pad_cnn_rgb_img_data']


In [86]:
# saving idtfd encoding to translate bow
tfidf_tokens = source_df["tfidf_tokens"]

In [87]:
# creating the training dataframe
train_df = pd.DataFrame(source_df, columns=keep_columns)

In [88]:
# shuffling the stuff
train_df = train_df.sample(frac = 1)
source_df = None
df_columns = list(train_df.columns)

In [89]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, s0132V1962 to s0097V1962r
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   std_cat_creator-place     59 non-null     object
 1   std_cat_details           59 non-null     object
 2   std_dvec_tokens           59 non-null     object
 3   std_pad_cnn_rgb_img_data  59 non-null     object
dtypes: object(4)
memory usage: 2.3+ KB


In [90]:
# getting the column with the relevant data to train
pad_regex = u"^std_pad_"
padimg_col = get_keeper_cols(df_columns, pad_regex)
padimg_col = padimg_col[0]
print("Padded image column in dataframe: ", str(padimg_col))

Padded image column in dataframe:  std_pad_cnn_rgb_img_data


In [91]:
# getting the column with the relevant data to train
dvec_regex = u"^std_dvec"
dvector_col = get_keeper_cols(df_columns, dvec_regex)
dvector_col = dvector_col[0]
print("Dense vector column in dataframe: ", str(dvector_col))

Dense vector column in dataframe:  std_dvec_tokens


In [92]:
# fix column data type
work_corpus = train_df[dvector_col]
work_corpus = format_dvector(work_corpus)

In [93]:
# changing type in dataframe
train_df[dvector_col] = work_corpus
work_corpus = None

In [94]:
# padding training data according to max length of text corpus
pad_prefix = "pad_"
recurrent_prefix = "lstm_"

train_df = padding_corpus(train_df, dvector_col, pad_prefix)

Original txt shape (59,)
Padded txt shape (59, 142)


In [95]:
regular_img_col = "std_" + work_imgt + "_img" + "_data"
padded_img_col = "std_" + pad_prefix + conv_prefix + work_imgt + "_img" + "_data"
padded_col_dvector = pad_prefix + dvector_col

In [96]:
# getting the columns with the relevant labels to predict
print(keep_columns)
cat_regex = u"^std_cat_"
labels_cols = get_keeper_cols(keep_columns, cat_regex)
print("Classifier trainable labels in dataframe: ", str(labels_cols))

# updating dataframe with hot/concatenated categories
tcat_label_col = "std_cat_labels"
print("categories heat column:", tcat_label_col)
train_df = heat_categories(train_df, labels_cols, tcat_label_col)

['std_cat_creator-place', 'std_cat_details', 'std_dvec_tokens', 'std_pad_cnn_rgb_img_data']
Classifier trainable labels in dataframe:  ['std_cat_creator-place', 'std_cat_details']
categories heat column: std_cat_labels


In [97]:
# getting the columns with the relevant labels to predict
print(keep_columns)
labels_cols = [i for i in keep_columns if re.search(u"^std_cat_", i)]
print("Trainable labels columns in dataframe: ", str(labels_cols))

labels_data = train_df[labels_cols]
labels_concat = list()

# concatenating all category labels from dataframe
for index, row in labels_data.iterrows():
    row = concat_labels(row, labels_cols)
    labels_concat.append(row)

['std_cat_creator-place', 'std_cat_details', 'std_dvec_tokens', 'std_pad_cnn_rgb_img_data']
Trainable labels columns in dataframe:  ['std_cat_creator-place', 'std_cat_details']


In [98]:
text_lstm_col = padded_col_dvector
print(text_lstm_col)

pad_std_dvec_tokens


In [99]:
working_img_col = padded_img_col
# working_img_col = regular_img_col
print(working_img_col)

std_pad_cnn_rgb_img_data


In [100]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, s0132V1962 to s0097V1962r
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   std_cat_creator-place     59 non-null     object
 1   std_cat_details           59 non-null     object
 2   std_dvec_tokens           59 non-null     object
 3   std_pad_cnn_rgb_img_data  59 non-null     object
 4   pad_std_dvec_tokens       59 non-null     object
 5   std_cat_labels            59 non-null     object
dtypes: object(6)
memory usage: 3.2+ KB


In [101]:
gc.collect()

60

In [102]:
# creating Train/Test sample
# getting the X, y to train, as is autoencoder both are the same
og_shape = train_df[working_img_col][0].shape# y[0].shape
X_img_len = train_df[working_img_col].shape[0] #y.shape[0]
print(X_img_len, og_shape)

X_img = None

for img in train_df[working_img_col]:

    if X_img is None:
        img = np.expand_dims(img, axis=0)
        X_img = img
    else:
        img = np.expand_dims(img, axis=0)
        X_img = np.concatenate((X_img, img), axis=0)

print("final X_img shape", X_img.shape)
# y.shape = (1899, 800, 800, 3)

59 (256, 256, 3)
final X_img shape (59, 256, 256, 3)


In [103]:
print(type(X_img[0]))
print(type(X_img[0][0]))
print(X_img[1].shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(256, 256, 3)


In [104]:
if len(X_img.shape) == 3:
    X_img = X_img[:,:,:,np.newaxis]

In [105]:
# y = train_df[working_img_col]
# y = np.expand_dims(y, axis=0)
y_labels = np.asarray([np.asarray(j, dtype="object") for j in train_df[tcat_label_col]], dtype="object")
print("y shape", y_labels.shape)

y shape (59, 16)


In [106]:
y = np.ones((y_labels.shape[0],1)).astype("float32")
print("y shape", y.shape)

y shape (59, 1)


In [107]:
print("y classification category")
print(type(y[0]))
print(type(y[0][0]))
print(y[1].shape)

print("y labels category")
print(type(y_labels[0]))
print(type(y_labels[0][0]))
print(y_labels[1].shape)

y classification category
<class 'numpy.ndarray'>
<class 'numpy.float32'>
(1,)
y labels category
<class 'numpy.ndarray'>
<class 'float'>
(16,)


In [108]:
# creating Train/Test sample
# getting the X, y to train, as is autoencoder both are the same
X_txt = np.asarray([np.asarray(i, dtype="object") for i in train_df[text_lstm_col]], dtype="object")
# X = np.array(train_df[text_lstm_col]).astype("object")
# X = train_df[text_lstm_col]
print("final X_lstm shape", X_txt.shape)

final X_lstm shape (59, 142)


In [109]:
print(type(X_txt[0]))
print(type(X_txt[0][0]))
print(X_txt[1].shape)

<class 'numpy.ndarray'>
<class 'float'>
(142,)


In [110]:
# timestep is the memory of what i read, this is the longest sentence I can remember in the short term
# neet to look for the best option, in small the max is 15
timesteps = 15

# features is the max length in the corpus, after padding!!!!
features = X_txt[0].shape[0]
print(timesteps, features)

15 142


In [111]:
X_txt = format_corpus(X_txt, timesteps, features)

(43, 15, 1, 142)
(43, 15, 142)


In [112]:
print(X_txt.shape)

(43, 15, 142)


In [113]:
diff_txt = y.shape[0] - X_txt.shape[0]
print(diff_txt)

16


In [114]:
Xa = X_txt[-diff_txt:]
X_txt = np.append(X_txt, Xa, axis=0)
print(X_txt.shape)
Xa = None

(59, 15, 142)


In [115]:
print(X_txt.shape)
print(X_img.shape)
print(y.shape)
print(y_labels.shape)

(59, 15, 142)
(59, 256, 256, 3)
(59, 1)
(59, 16)


In [116]:
print(X_txt[0].shape)
print(X_img[0].shape)
print(y[0].shape)
print(y_labels[0].shape)
txt_og_shape = X_txt[0].shape
img_og_shape = X_img[0].shape
cat_og_shape = y[0].shape
lab_og_shape = y_labels[0].shape

(15, 142)
(256, 256, 3)
(1,)
(16,)


In [117]:
# Xt = X_txt # np.array(X).astype("object")
# Xi = X_img
# yt = y # np.array(y).astype("object")
# # ya = y[0:timesteps]
# train_df = None

In [118]:
gc.collect()

75

# ML Model Definition

## Image GAN

In [119]:
# convolutional generator for images
def create_img_generator(latent_dims, model_cfg):

    # MODEL CONFIG
    # def of the latent space size for the input
    latent_features = model_cfg.get("latent_features")
    latent_filters = model_cfg.get("latent_filters")
    latent_dense = latent_features*latent_features*latent_filters
    # latent_input = latent_shape[0]*latent_shape[1]
    latent_n = model_cfg.get("latent_img_size")
    in_lyr_act = model_cfg.get("input_lyr_activation")
    # latent img shape
    latent_img_shape = model_cfg.get("latent_img_shape")

    # hidden layer config
    filters = model_cfg.get("filters")
    ksize = model_cfg.get("kernel_size")
    stsize = model_cfg.get("stride")
    pad = model_cfg.get("padding")
    hid_lyr_act = model_cfg.get("hidden_lyr_activation")
    hid_ldrop = model_cfg.get("gen_dropout_rate")
    mval = model_cfg.get("mask_value")
    rs = model_cfg.get("return_sequences")
    lstm_units = model_cfg.get("lstm_neurons")

    # output layer condig
    out_filters = model_cfg.get("output_filters")
    out_ksize = model_cfg.get("output_kernel_size")
    out_stsize = model_cfg.get("output_stride")
    out_pad = model_cfg.get("output_padding")
    img_shape = model_cfg.get("output_shape")
    out_lyr_act = model_cfg.get("output_lyr_activation")

    # LAYER CREATION
    # input layer
    in_latent = Input(shape=latent_dims, name="ImgGenIn")

    # flatting latent shape
    # lyr1 = Flatten(name="ImgGenFlat_1")(in_latent)

    # dense layer
    lyr1 = Dense(latent_dense, 
                activation=hid_lyr_act, 
                name="ImgGenDense_1")(in_latent)
    
    # reshape layer 1D-> 2D (rbg image)
    lyr2 = Reshape(latent_img_shape, name="ImgGenReshape_3")(lyr1)

    # transpose conv2D layer
    lyr3 = Conv2DTranspose(filters, kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgGenConv2D_4")(lyr2)

    # batch normalization + drop layers to avoid overfit
    lyr4 = BatchNormalization(name="ImgGenBN_5")(lyr3)
    lyr5 = Dropout(hid_ldrop, name="ImgGenDrop_6")(lyr4)


    # transpose conv2D layer
    lyr6 = Conv2DTranspose(int(filters/2), kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgGenConv2D_7")(lyr5)

    # batch normalization + drop layers to avoid overfit
    lyr7 = BatchNormalization(name="ImgGenBN_8")(lyr6)
    lyr8 = Dropout(hid_ldrop, name="ImgGenDrop_9")(lyr7)

    # transpose conv2D layer
    lyr9 = Conv2DTranspose(int(filters/4), kernel_size=ksize, 
                            strides=out_stsize, activation=out_lyr_act, 
                            padding=out_pad, name="ImgGenConv2D_10")(lyr8)

    # batch normalization + drop layers to avoid overfit
    lyr10 = BatchNormalization(name="ImgGenBN_11")(lyr9)
    lyr11 = Dropout(hid_ldrop, name="ImgGenDrop_12")(lyr10)

    # transpose conv2D layer
    lyr12 = Conv2DTranspose(int(filters/8), kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgGenConv2D_13")(lyr11)

    # batch normalization + drop layers to avoid overfit
    lyr13 = BatchNormalization(name="ImgGenBN_14")(lyr12)
    lyr14 = Dropout(hid_ldrop, name="ImgGenDrop_15")(lyr13)

    # transpose conv2D layer
    lyr15 = Conv2DTranspose(int(filters/16), kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgGenConv2D_16")(lyr14)

    # batch normalization + drop layers to avoid overfit
    lyr16 = BatchNormalization(name="ImgGenBN_17")(lyr15)
    lyr17 = Dropout(hid_ldrop, name="ImgGenDrop_18")(lyr16)

    # transpose conv2D layer
    lyr18 = Conv2DTranspose(int(filters/32), kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgGenConv2D_19")(lyr17)

    # batch normalization + drop layers to avoid overfit
    lyr19 = BatchNormalization(name="ImgGenBN_20")(lyr18)
    lyr20 = Dropout(hid_ldrop, name="ImgGenDrop_21")(lyr19)

    # output layer
    out_img = Conv2D(out_filters, kernel_size=out_ksize, 
                        strides=out_stsize, activation=out_lyr_act, 
                        padding=out_pad, input_shape=img_shape, 
                        name="ImgGenOut")(lyr20)

    # MODEL DEFINITION
    model = Model(inputs=in_latent, outputs=out_img)
    return model

In [120]:
# convolutional discriminator for images
def create_img_discriminator(img_shape, model_cfg):

    # MODEL CONFIG
    # input layer config, image classification
    in_lyr_act = model_cfg.get("input_lyr_activation")
    in_filters = model_cfg.get("input_filters")
    in_ksize = model_cfg.get("input_kernel_size")
    in_stsize = model_cfg.get("input_stride")
    in_pad = model_cfg.get("input_padding")

    # hidden layer config
    filters = model_cfg.get("filters")
    ksize = model_cfg.get("kernel_size")
    stsize = model_cfg.get("stride")
    pad = model_cfg.get("padding")
    hid_lyr_act = model_cfg.get("hidden_lyr_activation")
    hid_ldrop = model_cfg.get("dis_dropout_rate")
    # mid neuron size
    mid_disn = model_cfg.get("mid_dis_neurons")
    hid_cls_act = model_cfg.get("dense_cls_activation")

    # output layer condig
    out_nsize = model_cfg.get("output_dis_neurons")
    out_lyr_act = model_cfg.get("output_lyr_activation")

    # LAYER CREATION
    # input layer
    in_img = Input(shape=img_shape, name="DisImgIn")

    # DISCRIMINATOR LAYERS
    # intermediate conv layer
    lyr1 = Conv2D(int(in_filters/64), kernel_size=in_ksize, 
                    padding=in_pad, activation=in_lyr_act, 
                    strides=in_stsize, name="ImgDisConv2D_1")(in_img)

    # intermediate conv layer
    lyr2 = Conv2D(int(filters/32), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgDisConv2D_2")(lyr1)

    # batch normalization + drop layers to avoid overfit
    lyr3 = BatchNormalization(name="ImgDisBN_3")(lyr2)
    lyr4 = Dropout(hid_ldrop, name="ImgDisDrop_4")(lyr3)

    # intermediate conv layer
    lyr5 = Conv2D(int(filters/16), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgDisConv2D_5")(lyr4)

    # batch normalization + drop layers to avoid overfit
    lyr6 = BatchNormalization(name="ImgDisBN_6")(lyr5)
    lyr7 = Dropout(hid_ldrop, name="ImgDisDrop_7")(lyr6)

    # intermediate conv layer
    lyr8 = Conv2D(int(filters/8), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgDisConv2D_8")(lyr7)

    # batch normalization + drop layers to avoid overfit
    lyr9 = BatchNormalization(name="ImgDisBN_9")(lyr8)
    lyr10 = Dropout(hid_ldrop, name="ImgDisDrop_10")(lyr9)

    # intermediate conv layer
    lyr11 = Conv2D(int(filters/4), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgDisConv2D_11")(lyr10)

    # batch normalization + drop layers to avoid overfit
    lyr12 = BatchNormalization(name="ImgDisBN_12")(lyr11)
    lyr13 = Dropout(hid_ldrop, name="ImgDisDrop_13")(lyr12)

    # intermediate conv layer
    lyr14 = Conv2D(int(filters/2), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgDisConv2D_14")(lyr13)

    # batch normalization + drop layers to avoid overfit
    lyr15 = BatchNormalization(name="ImgDisBN_15")(lyr14)
    lyr16 = Dropout(hid_ldrop, name="ImgDisDrop_16")(lyr15)


    # flatten from 2D to 1D
    lyr17 = Flatten(name="ImgDisFlat_17")(lyr16)

    # dense classifier layers
    lyr18 = Dense(int(mid_disn), activation=hid_cls_act, name="ImgDisDense_18")(lyr17)
    lyr19 = Dense(int(mid_disn/2), activation=hid_cls_act, name="ImgDisDense_19")(lyr18)
    # drop layer
    lyr20 = Dropout(hid_ldrop, name="ImgDisDrop_20")(lyr19)

    # dense classifier layers
    lyr21 = Dense(int(mid_disn/4), activation=hid_cls_act, name="ImgDisDense_21")(lyr20)
    lyr22 = Dense(int(mid_disn/8), activation=hid_cls_act, name="ImgDisDense_22")(lyr21)
    # drop layer
    lyr23 = Dropout(hid_ldrop, name="ImgDisDrop_23")(lyr22)

    # dense classifier layers
    lyr24 = Dense(int(mid_disn/16), activation=hid_cls_act, name="ImgDisDense_24")(lyr23)
    lyr25 = Dense(int(mid_disn/32), activation=hid_cls_act, name="ImgDisDense_25")(lyr24)

    # output layer
    out_cls = Dense(out_nsize, activation=out_lyr_act, name="ImgDisOut")(lyr25)

    # MODEL DEFINITION
    model = Model(inputs=in_img, outputs=out_cls)
    return model

In [121]:
def create_img_gan(gen_model, dis_model, gan_cfg):

    # getting GAN Config
    ls = gan_cfg.get("loss")
    opt = gan_cfg.get("optimizer")
    met = gan_cfg.get("metrics")

	# make weights in the discriminator not trainable
    dis_model.trainable = False
	# get noise and label inputs from generator model
    gen_noise = gen_model.input
    # get image output from the generator model
    gen_output = gen_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = dis_model(gen_output)
    # define gan model as taking noise and label and outputting a classification
    model = Model(gen_noise, gan_output)
    # compile model
    model.compile(loss=ls, optimizer=opt, metrics=met)
    # model.compile(loss=ls, optimizer=opt)
    return model

## Text GAN

In [122]:
# LSTM generator for text
def create_txt_generator(latent_shape, model_cfg):

    # MODEL CONFIG
    # def of the latent space size for the input
    # input layer config, latent txt space
    mval = model_cfg.get("mask_value")
    in_rs = model_cfg.get("input_return_sequences")
    in_lstm = model_cfg.get("input_lstm_neurons")
    in_lyr_act = model_cfg.get("input_lyr_activation")

    # hidden layer config
    latent_n = model_cfg.get("mid_gen_neurons")
    latent_reshape = model_cfg.get("latent_lstm_reshape")
    lstm_units = model_cfg.get("lstm_neurons")
    hid_lyr_act = model_cfg.get("hidden_lyr_activation")
    hid_ldrop = model_cfg.get("gen_dropout_rate")
    mem_shape = model_cfg.get("memory_shape")
    rs = model_cfg.get("hidden_return_sequences")

    # output layer condig
    txt_shape = model_cfg.get("output_neurons")
    out_lyr_act = model_cfg.get("output_lyr_activation")

    # LAYER CREATION
    # input layer
    in_latent = Input(shape=latent_shape, name="TxtGenIn")

    # masking input text
    lyr1 = Masking(mask_value=mval, input_shape=latent_shape, 
                    name = "TxtGenMask_1")(in_latent) # concat1

    # intermediate recurrent layer
    lyr2 = LSTM(in_lstm, activation=in_lyr_act, 
                    input_shape=latent_shape, 
                    return_sequences=in_rs, 
                    name="TxtGenLSTM_2")(lyr1)

    # batch normalization + drop layers to avoid overfit
    lyr3 = BatchNormalization(name="TxtGenBN_3")(lyr2)
    lyr4 = Dropout(hid_ldrop, name="TxtGenDrop_4")(lyr3)

    # flatten from 2D to 1D
    lyr5 = Flatten(name="TxtGenFlat_5")(lyr4)

    # dense layer
    lyr6 = Dense(latent_n, 
                activation=hid_lyr_act, 
                name="TxtGenDense_6")(lyr5)

    # reshape layer 1D-> 2D (rbg image)
    lyr7 = Reshape(latent_reshape, name="TxtGenReshape_7")(lyr6)

    # batch normalization + drop layers to avoid overfit
    lyr8 = BatchNormalization(name="TxtGenBN_8")(lyr7)
    lyr9 = Dropout(hid_ldrop, name="TxtGenDrop_9")(lyr8)

    # intermediate recurrent layer
    lyr10 = LSTM(int(lstm_units/4), activation=hid_lyr_act, 
                    input_shape=mem_shape, 
                    return_sequences=rs, 
                    name="TxtGenLSTM_10")(lyr9)

    # intermediate recurrent layer
    lyr11 = LSTM(int(lstm_units/2), activation=hid_lyr_act, 
                    input_shape=mem_shape, 
                    return_sequences=rs, 
                    name="TxtGenLSTM_11")(lyr10)

    # batch normalization + drop layers to avoid overfit
    lyr12 = BatchNormalization(name="TxtGenBN_12")(lyr11)
    lyr13 = Dropout(hid_ldrop, name="TxtGenDrop_13")(lyr12)

    # output layer, dense time sequential layer.
    lyr14 = LSTM(lstm_units, activation=hid_lyr_act, 
                    input_shape=mem_shape, 
                    return_sequences=rs, 
                    name="TxtGenDrop_14")(lyr13)

    out_txt = TimeDistributed(Dense(txt_shape, activation=out_lyr_act), name = "GenTxtOut")(lyr14)

    # model definition
    model = Model(inputs=in_latent, outputs=out_txt)

    return model

In [123]:
# LSTM discriminator for text
def create_txt_discriminator(txt_shape, model_cfg):

    # MODEL CONFIG
    # def of the latent space size for the input
    # input layer config, latent txt space
    mval = model_cfg.get("mask_value")
    in_rs = model_cfg.get("input_return_sequences")
    in_lstm = model_cfg.get("input_lstm_neurons")
    in_lyr_act = model_cfg.get("input_lyr_activation")

    # hidden layer config
    lstm_units = model_cfg.get("lstm_neurons")
    hid_lyr_act = model_cfg.get("hidden_lyr_activation")
    hid_ldrop = model_cfg.get("dis_dropout_rate")
    mem_shape = model_cfg.get("memory_shape")
    rs = model_cfg.get("hidden_return_sequences")

    # mid neuron size
    mid_disn = model_cfg.get("mid_dis_neurons")
    hid_cls_act = model_cfg.get("dense_cls_activation")

    # output layer condig
    out_nsize = model_cfg.get("output_dis_neurons")
    out_lyr_act = model_cfg.get("output_lyr_activation")

    # LAYER CREATION
    # input layer
    in_txt = Input(shape=txt_shape, name="DisTxtIn")

    # DISCRIMINATOR LAYERS
    # masking input text
    lyr1 = Masking(mask_value=mval, input_shape=txt_shape, 
                    name = "TxtDisMask_1")(in_txt) # concat1

    # input LSTM layer
    lyr2 = LSTM(in_lstm, activation=in_lyr_act, 
                    input_shape=txt_shape, 
                    return_sequences=in_rs, 
                    name="TxtDisLSTM_2")(lyr1)

    # batch normalization + drop layers to avoid overfit
    lyr3 = BatchNormalization(name="TxtDisBN_3")(lyr2)
    lyr4 = Dropout(hid_ldrop, name="TxtDisDrop_4")(lyr3)

    # intermediate LSTM layer
    lyr5 = LSTM(int(lstm_units/2), 
                activation=hid_lyr_act, 
                input_shape=mem_shape, 
                return_sequences=rs, 
                name="TxtDisLSTM_5")(lyr4)

    # intermediate LSTM layer
    lyr6 = LSTM(int(lstm_units/4), 
                activation=hid_lyr_act, 
                input_shape=mem_shape, 
                return_sequences=rs, 
                name="TxtDisLSTM_6")(lyr5)

    # batch normalization + drop layers to avoid overfit
    lyr7 = BatchNormalization(name="TxtDisBN_7")(lyr6)
    lyr8 = Dropout(hid_ldrop, name="TxtDisDrop_8")(lyr7)

    # flatten from 2D to 1D
    lyr9 = Flatten(name="TxtDisFlat_9")(lyr8)

    # dense classifier layers
    lyr10 = Dense(int(mid_disn), activation=hid_cls_act, name="TxtDisDense_10")(lyr9)
    lyr11 = Dense(int(mid_disn/2), activation=hid_cls_act, name="TxtDisDense_11")(lyr10)
    # drop layer
    lyr12 = Dropout(hid_ldrop, name="TxtDisDrop_12")(lyr11)

    # dense classifier layers
    lyr13 = Dense(int(mid_disn/4), activation=hid_cls_act, name="TxtDisDense_13")(lyr12)
    lyr14 = Dense(int(mid_disn/8), activation=hid_cls_act, name="TxtDisDense_14")(lyr13)
    # drop layer
    lyr15 = Dropout(hid_ldrop, name="TxtDisDrop_15")(lyr14)

    # dense classifier layers
    lyr16 = Dense(int(mid_disn/16), activation=hid_cls_act, name="TxtDisDense_16")(lyr15)
    lyr17 = Dense(int(mid_disn/32), activation=hid_cls_act, name="TxtDisDense_17")(lyr16)

    # output layer
    out_cls = Dense(out_nsize, activation=out_lyr_act, name="TxtDisOut")(lyr17)

    # MODEL DEFINITION
    model = Model(inputs=in_txt, outputs=out_cls)
    return model

In [124]:
def create_txt_gan(gen_model, dis_model, gan_cfg):

    # getting GAN Config
    ls = gan_cfg.get("loss")
    opt = gan_cfg.get("optimizer")
    met = gan_cfg.get("metrics")

    # make weights in the discriminator not trainable
    dis_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise = gen_model.input
    # get image output from the generator model
    gen_output = gen_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = dis_model(gen_output)
    # define gan model as taking noise and label and outputting a classification
    model = Model(gen_noise, gan_output)
    # compile model
    model.compile(loss=ls, optimizer=opt, metrics=met)

    return model

## Conditional Img GAN: CGAN-img

In [125]:
# convolutional generator for images
def create_img_cgenerator(latent_dims, n_labels, model_cfg):

    # MODEL CONFIG
    # config for conditional labels
    memory = model_cfg.get("memory")
    features = model_cfg.get("features")
    lbl_neurons = model_cfg.get("labels_neurons")
    lbl_ly_actf = model_cfg.get("labels_lyr_activation")
    hid_ldrop = model_cfg.get("gen_dropout_rate")

    # def of the latent space size for the input
    latent_features = model_cfg.get("latent_features")
    latent_filters = model_cfg.get("latent_filters")
    latent_dense = latent_features*latent_features*latent_filters
    latent_n = model_cfg.get("latent_img_size")
    in_lyr_act = model_cfg.get("input_lyr_activation")
    latent_img_shape = model_cfg.get("latent_img_shape")

    # hidden layer config
    filters = model_cfg.get("filters")
    ksize = model_cfg.get("kernel_size")
    stsize = model_cfg.get("stride")
    pad = model_cfg.get("padding")
    hid_lyr_act = model_cfg.get("hidden_lyr_activation")
    hid_ldrop = model_cfg.get("gen_dropout_rate")
    mval = model_cfg.get("mask_value")
    rs = model_cfg.get("return_sequences")
    lstm_units = model_cfg.get("lstm_neurons")

    # output layer condig
    out_filters = model_cfg.get("output_filters")
    out_ksize = model_cfg.get("output_kernel_size")
    out_stsize = model_cfg.get("output_stride")
    out_pad = model_cfg.get("output_padding")
    img_shape = model_cfg.get("output_shape")
    out_lyr_act = model_cfg.get("output_lyr_activation")

    # CONDITIONAL LABELS LAYERS
    # label input
    in_labels = Input(shape=(n_labels,), name="ImgCGenLblIn")
    # embedding categorical textual input
    cond1 = Embedding(memory, features, input_length=n_labels, name="ImgCGenLblEmb_1")(in_labels)

    # flat layer
    cond2 = Flatten(name="ImgCGenLblFlat_2")(cond1)
    # dense layers
    cond3 = Dense(int(lbl_neurons/2), activation=lbl_ly_actf, name="ImgCGenLblDense_3")(cond2)

    # batch normalization + drop layers to avoid overfit
    cond4 = BatchNormalization(name="ImgCGenLblBN_4")(cond3)
    cond5 = Dropout(hid_ldrop, name="ImgCGenLblDrop_5")(cond4)

    cond6 = Dense(lbl_neurons, activation=lbl_ly_actf, name="ImgCGenLblDense_6")(cond5)

    # reshape layer
    cond7 = Dense(latent_dims, activation=lbl_ly_actf, name="ImgCGenLblOut")(cond6)

    # GENERATOR DEFINITION
    # LAYER CREATION
    # input layer
    in_latent = Input(shape=latent_dims, name="ImgCGenIn")

    # concat generator layers + label layers
    lbl_concat = Concatenate(axis=-1, name="ImgCGenConcat")([in_latent, cond7])

    # dense layer
    lyr1 = Dense(latent_dense, 
                activation=hid_lyr_act, 
                name="ImgGenCDense_1")(lbl_concat)
                
    # batch normalization + drop layers to avoid overfit
    lyr2 = BatchNormalization(name="ImgCGenBN_2")(lyr1)
    lyr3 = Dropout(hid_ldrop, name="ImgCGenDrop_3")(lyr2)

    # dense layer
    lyr4 = Dense(latent_n, 
                activation=hid_lyr_act, 
                name="ImgCGenDense_4")(lyr3)
    
    # reshape layer 1D-> 2D (rbg image)
    lyr5 = Reshape(latent_img_shape, name="ImgCGenReshape_5")(lyr4)

    # transpose conv2D layer
    lyr6 = Conv2DTranspose(filters, kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgCGenConv2D_6")(lyr5)

    # batch normalization + drop layers to avoid overfit
    lyr7 = BatchNormalization(name="ImgCGenBN_7")(lyr6)
    lyr8 = Dropout(hid_ldrop, name="ImgCGenDrop_8")(lyr7)


    # transpose conv2D layer
    lyr9 = Conv2DTranspose(filters, kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgCGenConv2D_9")(lyr8)

    # transpose conv2D layer
    lyr10 = Conv2DTranspose(filters, kernel_size=ksize, 
                            strides=out_stsize, activation=out_lyr_act, 
                            padding=out_pad, name="ImgCGenConv2D_10")(lyr9)

    # batch normalization + drop layers to avoid overfit
    lyr11 = BatchNormalization(name="ImgCGenBN_11")(lyr10)
    lyr12 = Dropout(hid_ldrop, name="ImgCGenDrop_12")(lyr11)

    # transpose conv2D layer
    lyr13 = Conv2DTranspose(filters, kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgCGenConv2D_13")(lyr12)

    # output layer
    out_img = Conv2D(out_filters, kernel_size=out_ksize, 
                        strides=out_stsize, activation=out_lyr_act, 
                        padding=out_pad, input_shape=img_shape, 
                        name="ImgCGenOut")(lyr13)

    # MODEL DEFINITION
    model = Model(inputs=[in_latent, in_labels], outputs=out_img)
    return model

In [126]:
# convolutional discriminator for images
def create_img_cdiscriminator(img_shape, n_labels, model_cfg):

    # MODEL CONFIG
    # config for conditional labels
    memory = model_cfg.get("timesteps")
    features = model_cfg.get("max_features")
    lbl_neurons = model_cfg.get("labels_neurons")
    lbl_ly_actf = model_cfg.get("labels_lyr_activation")
    lbl_filters = model_cfg.get("labels_filters")
    lbl_ksize = model_cfg.get("labels_kernel_size")
    lbl_stsize = model_cfg.get("labels_stride")
    gen_reshape = model_cfg.get("labels_reshape")
    hid_ldrop = model_cfg.get("dis_dropout_rate")

    # input layer config, image classification
    in_lyr_act = model_cfg.get("input_lyr_activation")
    in_filters = model_cfg.get("input_filters")
    in_ksize = model_cfg.get("input_kernel_size")
    in_stsize = model_cfg.get("input_stride")
    in_pad = model_cfg.get("input_padding")

    # hidden layer config
    filters = model_cfg.get("filters")
    ksize = model_cfg.get("kernel_size")
    stsize = model_cfg.get("stride")
    pad = model_cfg.get("padding")
    hid_lyr_act = model_cfg.get("hidden_lyr_activation")
    hid_ldrop = model_cfg.get("dis_dropout_rate")
    # mid neuron size
    mid_disn = model_cfg.get("mid_dis_neurons")
    hid_cls_act = model_cfg.get("dense_cls_activation")

    # output layer condig
    out_nsize = model_cfg.get("output_dis_neurons")
    out_lyr_act = model_cfg.get("output_lyr_activation")

    # LABEL IMG LAYERS
    # label inpuy
    in_labels = Input(shape=(n_labels,), 
                        name="ImgCDisLblIn")

    # embedding categorical textual input
    cond1 = Embedding(memory, features, 
                        input_length=n_labels, 
                        name="ImgCDisLblEmb_1")(in_labels)

    # flat layer
    cond2 = Flatten(name="ImgCDisLblFlat_2")(cond1)
    cond3 = Dense(lbl_neurons, activation=lbl_ly_actf, 
                    name="ImgCDisLblDense_3")(cond2)
    
    # reshape layer
    cond4 = Reshape(gen_reshape, name="ImgCDisLblReshape_4")(cond3)

    # transpose conv2D layers
    cond5 = Conv2DTranspose(int(lbl_filters/8), kernel_size=ksize, 
                                strides=stsize, activation=lbl_ly_actf, 
                                padding=pad, name="ImgCDisLblConv2D_5")(cond4)

    # batch normalization + drop layers to avoid overfit
    cond6 = BatchNormalization(name="ImgCDisLblBN_6")(cond5)
    cond7 = Dropout(hid_ldrop, name="ImgCDisLblDrop_7")(cond6)

    # trnaspose conv2D layers
    cond8 = Conv2DTranspose(int(lbl_filters/4), kernel_size=ksize, 
                                strides=stsize, activation=lbl_ly_actf, 
                                padding=pad, name="ImgCDisLblConv2D_8")(cond7)

    # batch normalization + drop layers to avoid overfit
    cond9 = BatchNormalization(name="ImgCDisLblBN_9")(cond8)
    cond10 = Dropout(hid_ldrop, name="ImgCDisLblDrop_10")(cond9)

    # conditional layer output
    cond11 = Conv2DTranspose(img_shape[2], kernel_size=ksize, 
                                strides=stsize, activation=lbl_ly_actf, 
                                padding=pad, name="ImgCDisLblOut")(cond10)

    # LAYER CREATION
    # input layer
    in_img = Input(shape=img_shape, name="ImgCDisIn")

    lbl_concat = Concatenate(axis=-1, name="ImgCDisConcat")([in_img, cond11])

    # DISCRIMINATOR LAYERS
    # intermediate conv layer
    lyr1 = Conv2D(in_filters, kernel_size=in_ksize, 
                    padding=in_pad, activation=in_lyr_act, 
                    strides=in_stsize, name="ImgCDisConv2D_1")(lbl_concat)

    # intermediate conv layer
    lyr2 = Conv2D(int(filters/2), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgCDisConv2D_2")(lyr1)

    # batch normalization + drop layers to avoid overfit
    lyr3 = BatchNormalization(name="ImgCDisBN_3")(lyr2)
    lyr4 = Dropout(hid_ldrop, name="ImgCDisDrop_4")(lyr3)

    # intermediate conv layer
    lyr5 = Conv2D(int(filters/4), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgCDisConv2D_5")(lyr4)

    # intermediate conv layer
    lyr6 = Conv2D(int(filters/8), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgCDisConv2D_6")(lyr5)

    # batch normalization + drop layers to avoid overfit
    lyr7 = BatchNormalization(name="ImgCDisBN_7")(lyr6)
    lyr8 = Dropout(hid_ldrop, name="ImgCDisDrop_8")(lyr7)

    # flatten from 2D to 1D
    lyr9 = Flatten(name="ImgCDisFlat_9")(lyr8)

    # dense classifier layers
    lyr10 = Dense(int(mid_disn), activation=hid_cls_act, name="ImgCDisDense_10")(lyr9)
    lyr11 = Dense(int(mid_disn/2), activation=hid_cls_act, name="ImgCDisDense_11")(lyr10)
    # drop layer
    lyr12 = Dropout(hid_ldrop, name="ImgCDisDrop_12")(lyr11)

    # dense classifier layers
    lyr13 = Dense(int(mid_disn/4), activation=hid_cls_act, name="ImgCDisDense_13")(lyr12)
    lyr14 = Dense(int(mid_disn/8), activation=hid_cls_act, name="ImgCDisDense_14")(lyr13)
    # drop layer
    lyr15 = Dropout(hid_ldrop, name="ImgCDisDrop_15")(lyr14)

    # dense classifier layers
    lyr16 = Dense(int(mid_disn/16), activation=hid_cls_act, name="ImgCDisDense_16")(lyr15)
    lyr17 = Dense(int(mid_disn/32), activation=hid_cls_act, name="ImgCDisDense_17")(lyr16)

    # output layer
    out_cls = Dense(out_nsize, activation=out_lyr_act, name="ImgCDisOut")(lyr17)

    # MODEL DEFINITION
    model = Model(inputs=[in_img, in_labels], outputs=out_cls)
    return model

In [127]:
def create_img_cgan(gen_model, dis_model, gan_cfg):

    # getting GAN Config
    ls = gan_cfg.get("loss")
    opt = gan_cfg.get("optimizer")
    met = gan_cfg.get("metrics")

    # make weights in the discriminator not trainable
    dis_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_labels = gen_model.input
    # get image output from the generator model
    gen_output = gen_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = dis_model([gen_output, gen_labels])
    # define gan model as taking noise and label and outputting a classification
    gan_model = Model([gen_noise, gen_labels], gan_output)
    # compile model
    gan_model.compile(loss=ls, optimizer=opt, metrics=met)
    # cgan_model.compile(loss=gan_cfg[0], optimizer=gan_cfg[1])#, metrics=gan_cfg[2])
    return gan_model

## Multi GAN txt2img

In [128]:
# LSTM + Conv discriminator for image and text
# TODO need to implement this
def create_multi_discriminator(img_shape, txt_shape, model_cfg):

    # model definition
    model = Model(inputs=[in_img, in_txt], outputs=out_cls)

    return model

In [129]:
# TODO need to implement this
def create_multi_generator(img_shape, txt_shape, model_cfg):

    # model definition
    gen_model = Model(inputs=in_latent, outputs=[out_img, out_txt])
    return model

In [130]:
# TODO need to implement this
def create_multi_gan(gen_model, dis_model, gan_cfg):

    # getting GAN Config
    ls = gan_cfg.get("loss")
    opt = gan_cfg.get("optimizer")
    met = gan_cfg.get("metrics")

    # make weights in the discriminator not trainable
    cdis_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_labels = cgen_model.input
    # get image output from the generator model
    gen_output = cgen_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = dis_model([gen_output, gen_labels])
    # define gan model as taking noise and label and outputting a classification
    gan_model = Model([gen_noise, gen_labels], gan_output)
    # compile model
    gan_model.compile(loss=gan_cfg[0], optimizer=gan_cfg[1], metrics=gan_cfg[2])
    # cgan_model.compile(loss=gan_cfg[0], optimizer=gan_cfg[1])#, metrics=gan_cfg[2])
    return gan_model

## Multi CGAN txt2img

In [131]:
def create_multi_cgenerator(latent_dims, img_shape, txt_shape, n_labels, model_cfg):

    # MODEL CONFIG
    # config for conditional labels
    # print("=======================\n",model_cfg, "=====================")
    memory = model_cfg.get("memory")
    features = model_cfg.get("features")
    lbl_neurons = model_cfg.get("labels_neurons")
    lbl_ly_actf = model_cfg.get("labels_lyr_activation")
    hid_ldrop = model_cfg.get("gen_dropout_rate")

    # def of the latent space size for the input
    # input layer config, latent txt space
    latent_nimg = model_cfg.get("latent_img_size")
    latent_features = model_cfg.get("latent_features")
    latent_filters = model_cfg.get("latent_filters")
    latent_dense = latent_features*latent_features*latent_filters
    in_lyr_act = model_cfg.get("input_lyr_activation")
    latent_img_shape = model_cfg.get("latent_img_shape")
    mval = model_cfg.get("mask_value")
    in_rs = model_cfg.get("input_return_sequences")
    in_lstm = model_cfg.get("input_lstm_neurons")

    # hidden layer config
    filters = model_cfg.get("filters")
    ksize = model_cfg.get("kernel_size")
    stsize = model_cfg.get("stride")
    pad = model_cfg.get("padding")
    hid_lyr_act = model_cfg.get("hidden_lyr_activation")
    rs = model_cfg.get("return_sequences")
    lstm_units = model_cfg.get("lstm_neurons")
    latent_ntxt = model_cfg.get("mid_gen_neurons")
    latent_txt_shape = model_cfg.get("latent_lstm_reshape")
    mem_shape = model_cfg.get("memory_shape")
    rs = model_cfg.get("hidden_return_sequences")

    # output layer condig
    out_filters = model_cfg.get("output_filters")
    out_ksize = model_cfg.get("output_kernel_size")
    out_stsize = model_cfg.get("output_stride")
    out_pad = model_cfg.get("output_padding")
    img_shape = model_cfg.get("output_shape")
    out_lyr_act = model_cfg.get("output_lyr_activation")
    txt_shape = model_cfg.get("output_neurons")
    out_rs = model_cfg.get("output_return_sequences")

    # MODEL DEF
    # CONDITIONAL LABELS LAYERS FOR IMG + TXT
    # label input
    in_labels = Input(shape=(n_labels,), name="MultiCGenLblIn")
    # embedding categorical textual input
    cond1 = Embedding(memory, features, input_length=n_labels, name="MultiCGenLblEmb_1")(in_labels)

    # flat layer
    cond2 = Flatten(name="MultiCGenLblFlat_2")(cond1)

    # dense layers
    cond3 = Dense(int(lbl_neurons/2), activation=lbl_ly_actf, name="MultiCGenLblDense_3")(cond2)

    # batch normalization + drop layers to avoid overfit
    cond4 = BatchNormalization(name="MultiCGenLblBN_4")(cond3)
    cond5 = Dropout(hid_ldrop, name="MultiCGenLblDrop_5")(cond4)

    # dense layer
    cond6 = Dense(lbl_neurons, activation=lbl_ly_actf, name="MultiCGenLblDense_6")(cond5)

    # reshape layer
    cond7 = Dense(latent_dims, activation=lbl_ly_actf, name="MultiCGenLblDense_7")(cond6)

    # GENERATOR DEFINITION
    #LATENT LAYER CREATION
    # input layer
    in_latent = Input(shape=latent_dims, name="MultiCGenIn")

    # concat generator layers + label layers
    lbl_concat = Concatenate(axis=-1, name="MultiCGenConcat")([in_latent, cond7])

    # Dense input layer
    olyr1 = Dense(latent_dense, 
                    activation=hid_lyr_act, 
                    name= "MultiCGenDense_1")(lbl_concat) # contat!!!!

    # dense to txt latent space
    olyr2 = Dense(latent_ntxt, 
                activation=hid_lyr_act, 
                name="MultiCGenDense_2")(olyr1)

    # batch normalization + drop layers to avoid overfit
    olyr3 = BatchNormalization(name="MultiCGenBN_3")(olyr2)
    olyr4 = Dropout(hid_ldrop, name="MultiCGenDrop_4")(olyr3)

    # reshape for LSTM layer
    olyr5 = Reshape(latent_txt_shape, name="MultiCGenReshape_5")(olyr4)

    # intermediate recurrent layer for gen memory
    olyr6 = LSTM(lstm_units, activation=in_lyr_act, 
                    input_shape=latent_txt_shape, 
                    return_sequences=rs, name="MultiCGenLSTM_6")(olyr5)

    # batch normalization + drop layers to avoid overfit
    olyr7 = BatchNormalization(name="MultiCGenBN_7")(olyr6)
    olyr8 = Dropout(hid_ldrop, name="MultiCGenDrop_8")(olyr7)

    # flaten for latent img + txt space 2D to 1D
    olyr9 = Flatten(name="ImgTxtCGenFlat_9")(olyr8)

    # dense layer for img reshape
    olyr10 = Dense(latent_nimg, 
                activation=hid_lyr_act, 
                name="MultiCGenDense_10")(olyr9)
    
    # dense layer for txt reshape
    olyr11 = Dense(latent_ntxt, 
                activation=hid_lyr_act, 
                name="MultiCGenDense_11")(olyr9)

    # IMG GENERATOR
    # reshape layer 1D-> 2D (rbg image)
    ilyr5 = Reshape(latent_img_shape, name="ImgCGenReshape_5")(olyr10)

    # transpose conv2D layer
    ilyr6 = Conv2DTranspose(filters, kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgCGenConv2D_6")(ilyr5)

    # batch normalization + drop layers to avoid overfit
    ilyr7 = BatchNormalization(name="ImgCGenBN_7")(ilyr6)
    ilyr8 = Dropout(hid_ldrop, name="ImgCGenDrop_8")(ilyr7)

    # transpose conv2D layer
    ilyr9 = Conv2DTranspose(int(filters/2), kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgCGenConv2D_9")(ilyr8)

    # transpose conv2D layer
    ilyr10 = Conv2DTranspose(int(filters/4), kernel_size=ksize, 
                            strides=out_stsize, activation=out_lyr_act, 
                            padding=out_pad, name="ImgCGenConv2D_10")(ilyr9)

    # batch normalization + drop layers to avoid overfit
    ilyr11 = BatchNormalization(name="ImgCGenBN_11")(ilyr10)
    ilyr12 = Dropout(hid_ldrop, name="ImgCGenDrop_12")(ilyr11)

    # transpose conv2D layer
    ilyr13 = Conv2DTranspose(int(filters/8), kernel_size=ksize, 
                            strides=stsize, activation=hid_lyr_act, 
                            padding=pad, name="ImgCGenConv2D_13")(ilyr12)

    # output layer
    out_img = Conv2D(out_filters, kernel_size=out_ksize, 
                        strides=out_stsize, activation=out_lyr_act, 
                        padding=out_pad, input_shape=img_shape, 
                        name="ImgCGenOut")(ilyr13)

    # TXT GENERATOR
    # reshape layer 1D-> 2D (descriptive txt)
    tlyr6 = Reshape(latent_txt_shape, name="TxtCGenReshape_6")(olyr11)

    # batch normalization + drop layers to avoid overfit
    tlyr7 = BatchNormalization(name="TxtCGenBN_7")(tlyr6)
    tlyr8 = Dropout(hid_ldrop, name="TxtCGenDrop_8")(tlyr7)

    # intermediate recurrent layer
    tlyr9 = LSTM(int(lstm_units/4), activation=hid_lyr_act, 
                    input_shape=mem_shape, 
                    return_sequences=rs, 
                    name="TxtCGenLSTM_9")(tlyr8)

    # intermediate recurrent layer
    tlyr10 = LSTM(int(lstm_units/2), activation=hid_lyr_act, 
                    input_shape=mem_shape, 
                    return_sequences=rs, 
                    name="TxtCGenLSTM_10")(tlyr9)

    # batch normalization + drop layers to avoid overfit
    tlyr11 = BatchNormalization(name="TxtCGenBN_11")(tlyr10)
    tlyr12 = Dropout(hid_ldrop, name="TxtCGenDrop_12")(tlyr11)

    # output layer, dense time sequential layer.
    tlyr13 = LSTM(lstm_units, activation=hid_lyr_act, 
                    input_shape=mem_shape, 
                    return_sequences=rs, 
                    name="TxtCGenLSTM_13")(tlyr12)

    out_txt = TimeDistributed(Dense(txt_shape, activation=out_lyr_act), name = "TxtCGenOut")(tlyr13)

    # MODEL DEFINITION
    model = Model(inputs=[in_latent, in_labels], outputs=[out_img, out_txt])

    return model

In [132]:
# LSTM + Conv conditianal discriminator for text and images
def create_multi_cdiscriminator(img_shape, txt_shape, n_labels, model_cfg):

    # MODEL CONFIG
    # config for txt + img conditional labels
    memory = model_cfg.get("timesteps")
    features = model_cfg.get("max_features")
    lbl_neurons = model_cfg.get("labels_neurons")
    lbl_ly_actf = model_cfg.get("labels_lyr_activation")
    lbl_filters = model_cfg.get("labels_filters")
    lbl_ksize = model_cfg.get("labels_kernel_size")
    lbl_stsize = model_cfg.get("labels_stride")
    lbl_lstm = model_cfg.get("labels_lstm_neurons")
    lbl_rs = model_cfg.get("labels_return_sequences")
    dis_img_reshape = model_cfg.get("labels_img_reshape")
    dis_txt_reshape = model_cfg.get("labels_txt_reshape")

    # input layer config for image classification
    in_lyr_act = model_cfg.get("input_lyr_activation")
    in_filters = model_cfg.get("input_filters")
    in_ksize = model_cfg.get("input_kernel_size")
    in_stsize = model_cfg.get("input_stride")
    in_pad = model_cfg.get("input_padding")

    # input layer config for txt classification
    mval = model_cfg.get("mask_value")
    in_rs = model_cfg.get("input_return_sequences")
    in_lstm = model_cfg.get("input_lstm_neurons")
    in_lyr_act = model_cfg.get("input_lyr_activation")

    # encoding hidden layer config for image classification
    filters = model_cfg.get("filters")
    ksize = model_cfg.get("kernel_size")
    stsize = model_cfg.get("stride")
    pad = model_cfg.get("padding")
    hid_lyr_act = model_cfg.get("hidden_lyr_activation")
    hid_ldrop = model_cfg.get("dis_dropout_rate")

    # encoding hidden layer config for text classification
    lstm_units = model_cfg.get("lstm_neurons")
    mem_shape = model_cfg.get("memory_shape")
    rs = model_cfg.get("hidden_return_sequences")

    # mid classification config
    mid_disn = model_cfg.get("mid_dis_neurons")
    hid_cls_act = model_cfg.get("dense_cls_activation")

    # output layer config
    out_nsize = model_cfg.get("output_dis_neurons")
    out_lyr_act = model_cfg.get("output_lyr_activation")

    # IMG/TXT LABELS CONDITIONAL LAYERS
    # labels input
    in_labels = Input(shape=(n_labels,), 
                        name="MultiCDisLblIn")

    # embedding categorical textual input
    cond1 = Embedding(memory, features, 
                        input_length=n_labels, 
                        name="MultiCDisLblEmb_1")(in_labels)

    # flat layer
    cond2 = Flatten(name="MultiCDisLblFlat_2")(cond1)
    # img dense layer
    cond3 = Dense(lbl_neurons, activation=lbl_ly_actf, 
                    name="MultiCDisLblDense_3")(cond2)
    
    # image reshape layer
    cond4i = Reshape(dis_img_reshape, name="ImgCDisLblReshape_4")(cond3)

    # txt dense layer
    cond5 = Dense(int(memory*features), activation=lbl_ly_actf, 
                    name="MultiCDisLblDense_5")(cond2)
    # txt reshape layer
    cond6t = Reshape(dis_txt_reshape, name="TxtCDisLblReshape_6")(cond5)

    # transpose conv2D layer for img
    cond7 = Conv2DTranspose(int(lbl_filters), kernel_size=ksize, 
                                strides=stsize, activation=lbl_ly_actf, 
                                padding=pad, name="ImgCDisLblConv2D_7")(cond4i)

    # batch normalization + drop layers to avoid overfit for img
    cond8 = BatchNormalization(name="ImgCDisLblBN_8")(cond7)
    cond9 = Dropout(hid_ldrop, name="ImgCDisLblDrop_9")(cond8)

    # transpose conv2D layers for img
    cond10 = Conv2DTranspose(int(lbl_filters), kernel_size=ksize, 
                                strides=stsize, activation=lbl_ly_actf, 
                                padding=pad, name="ImgCDisLblConv2D_10")(cond9)

    # batch normalization + drop layers to avoid overfit for img
    cond11 = BatchNormalization(name="ImgCDisLblBN_11")(cond10)
    cond12 = Dropout(hid_ldrop, name="ImgCDisLblDrop_12")(cond11)

    # conditional layer output for img
    cond13 = Conv2DTranspose(img_shape[2], kernel_size=ksize, 
                                strides=stsize, activation=lbl_ly_actf, 
                                padding=pad, name="ImgCDisLblOut")(cond12)

    # intermediate LSTM layer for text
    cond14 = LSTM(int(lbl_lstm), 
                activation=lbl_ly_actf, 
                input_shape=mem_shape, 
                return_sequences=lbl_rs, 
                name="TxtCDisLblLSTM_14")(cond6t)

    # batch normalization + drop layers to avoid overfit for img
    cond15 = BatchNormalization(name="TxtCDisLblBN_15")(cond14)
    cond16 = Dropout(hid_ldrop, name="TxtCDisLblDrop_16")(cond15)

    # intermediate LSTM layer for text
    cond17 = LSTM(lbl_lstm, 
                activation=lbl_ly_actf, 
                input_shape=mem_shape, 
                return_sequences=lbl_rs, 
                name="TxtCDisLblLSTM_17")(cond16)

    # LAYER CREATION
    # input layer
    in_img = Input(shape=img_shape, name="CDisImgIn")

    concat_img = Concatenate(axis=-1, name="ImgCDisConcat_19")([in_img, cond13])

    # DISCRIMINATOR LAYERS
    # intermediate conv layer
    lyr1 = Conv2D(in_filters, kernel_size=in_ksize, 
                    padding=in_pad, activation=in_lyr_act, 
                    strides=in_stsize, name="ImgCDisConv2D_20")(concat_img)

    # intermediate conv layer
    lyr2 = Conv2D(int(filters/2), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgCDisConv2D_21")(lyr1)

    # batch normalization + drop layers to avoid overfit
    lyr3 = BatchNormalization(name="ImgCDisBN_22")(lyr2)
    lyr4 = Dropout(hid_ldrop, name="ImgCDisDrop_23")(lyr3)

    # intermediate conv layer
    lyr5 = Conv2D(int(filters/4), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgCDisConv2D_24")(lyr4)

    # intermediate conv layer
    lyr6 = Conv2D(int(filters/8), kernel_size=ksize, 
                    padding=pad, activation=hid_lyr_act, 
                    strides=stsize, name="ImgCDisConv2D_25")(lyr5)

    # batch normalization + drop layers to avoid overfit
    lyr7 = BatchNormalization(name="ImgCDisBN_26")(lyr6)
    lyr8 = Dropout(hid_ldrop, name="ImgCDisDrop_27")(lyr7)

    # flatten from 2D to 1D
    lyr9 = Flatten(name="ImgCDisFlat_28")(lyr8)

    #TXT DISCRIMINATOR
    # LAYER CREATION
    # input layer
    in_txt = Input(shape=txt_shape, name="CDisTxtIn")

    # concat txt input with labels conditional
    concat_txt = Concatenate(axis=-1, name="TxtCDisConcat_30")([in_txt, cond17])

    # DISCRIMINATOR LAYERS
    # masking input text
    lyr10 = Masking(mask_value=mval, input_shape=txt_shape, 
                    name = "TxtCDisMask_31")(concat_txt) # concat1

    # input LSTM layer
    lyr11 = LSTM(in_lstm, activation=in_lyr_act, 
                    input_shape=txt_shape, 
                    return_sequences=in_rs, 
                    name="TxtCDisLSTM_32")(lyr10)

    # batch normalization + drop layers to avoid overfit
    lyr12 = BatchNormalization(name="TxtCDisBN_33")(lyr11)
    lyr13 = Dropout(hid_ldrop, name="TxCtDisDrop_34")(lyr12)

    # intermediate LSTM layer
    lyr14 = LSTM(int(lstm_units/2), 
                activation=hid_lyr_act, 
                input_shape=mem_shape, 
                return_sequences=rs, 
                name="TxtCDisLSTM_35")(lyr13)

    # intermediate LSTM layer
    lyr15 = LSTM(int(lstm_units/4), 
                activation=hid_lyr_act, 
                input_shape=mem_shape, 
                return_sequences=rs, 
                name="TxtCDisLSTM_36")(lyr14)

    # batch normalization + drop layers to avoid overfit
    lyr16 = BatchNormalization(name="TxtCDisBN_37")(lyr15)
    lyr17 = Dropout(hid_ldrop, name="TxtCDisDrop_38")(lyr16)

    # flatten from 2D to 1D
    lyr18 = Flatten(name="TxtCDisFlat_39")(lyr17)

    # concat img encoding + txt encoding
    concat_encoding = Concatenate(axis=-1, name="MultiCDisConcat_40")([lyr18, lyr9])

    # dense classifier layers
    lyr19 = Dense(int(mid_disn), activation=hid_cls_act, name="MultiCDisDense_41")(concat_encoding)
    lyr20 = Dense(int(mid_disn/2), activation=hid_cls_act, name="MultiCDisDense_42")(lyr19)
    # drop layer
    lyr21 = Dropout(hid_ldrop, name="MultiCDisDrop_43")(lyr20)

    # dense classifier layers
    lyr22 = Dense(int(mid_disn/4), activation=hid_cls_act, name="MultiCDisDense_44")(lyr21)
    lyr23 = Dense(int(mid_disn/8), activation=hid_cls_act, name="MultiCDisDense_45")(lyr22)
    # drop layer
    lyr24 = Dropout(hid_ldrop, name="MultiCDisDrop_46")(lyr23)

    # dense classifier layers
    lyr25 = Dense(int(mid_disn/16), activation=hid_cls_act, name="MultiCDisDense_47")(lyr24)
    lyr26 = Dense(int(mid_disn/32), activation=hid_cls_act, name="MultiCDisDense_48")(lyr25)

    # output layer
    out_cls = Dense(out_nsize, activation=out_lyr_act, name="MultiCDisOut")(lyr26)

    # model definition
    model = Model(inputs=[in_img, in_txt, in_labels], outputs=out_cls)

    return model

In [133]:
def create_multi_cgan(gen_model, dis_model, gan_cfg):

    # getting GAN Config
    ls = gan_cfg.get("loss")
    opt = gan_cfg.get("optimizer")
    met = gan_cfg.get("metrics")

    # make weights in the discriminator not trainable
    dis_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_labels = gen_model.input
    # get image output from the generator model
    gen_img, gen_txt = gen_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = dis_model([gen_img, gen_txt, gen_labels])
    # define gan model as taking noise and label and outputting a classification
    gan_model = Model([gen_noise, gen_labels], gan_output)
    # compile model
    gan_model.compile(loss=ls, optimizer=opt, metrics=met)
    # cgan_model.compile(loss=gan_cfg[0], optimizer=gan_cfg[1])#, metrics=gan_cfg[2])
    return gan_model

## ML Models Configuration
### GAN-img definition

In [134]:
REF_KERNEL_SIZE =(4,4) #(4,4)
REF_FILTERS = 128*8
REF_LSTM = 400

# common variables for the models
# input common vars
input_filters = REF_FILTERS
input_kernel_size = REF_KERNEL_SIZE
input_stride = (2,2)
input_padding = "same"
input_lstm_neurons = REF_LSTM
mask_value = 0.0
input_return_sequences = True

# latent and conditional label common vars
# def of the latent space size for the input
latent_features = 8 # 5 # model_cfg.get("latent_features")
latent_filters = REF_FILTERS # 128 # model_cfg.get("latent_filters")
latent_lstm_reshape = X_txt[0].shape
memory_shape = X_txt[0].shape
labels_neurons = timesteps*X_txt.shape[2]
latent_img_size = 8*8*REF_FILTERS # 50*50*8 # 32*32*3, # 5*5*128 #
latent_img_shape = (8,8,REF_FILTERS) # (50,50,8) # (32,32,3), # (5,5,128) # 
labels_img_neurons =  8*8*REF_FILTERS # 50*50*3
labels_filters = REF_FILTERS
labels_kernel_size = REF_KERNEL_SIZE
labels_stride = (2,2)
labels_reshape = (50,50,3)
labels_lstm_neurons = REF_LSTM
labels_return_sequences = True
labels_txt_reshape = X_txt[0].shape


# hidden common vars
lstm_neurons = REF_LSTM
filters = REF_FILTERS
kernel_size = (4,4)
stride = (2,2)
padding = "same"
gen_dropout_rate = 0.3
mid_txt_gen_neurons = X_txt.shape[1]*X_txt.shape[2]
hidden_return_sequences = True
dis_dropout_rate = 0.2
mid_dis_neurons = 50*50*2 # 32*32*3,

# output common vars
output_neurons = X_txt.shape[2]
output_txt_shape = X_txt[0].shape
output_gen_lyr_activation = "tanh"
output_return_sequences = True
output_filters = X_img[0].shape[2]
output_kernel_size = (3,3)
output_stride = (1,1)
output_img_shape = X_img[0].shape
output_dis_neurons = 1
output_dis_lyr_activation = "sigmoid"

# looss qnd activation functions functions
DIS_OPTI_REF = Adam(learning_rate=0.00005, beta_1=0.5)
GEN_OPTI_REF = Adam(learning_rate=0.0002, beta_1=0.5)
LOSS_REF = "binary_crossentropy"
ACC_REF = ["accuracy"]
ACT_REF = LeakyReLU(alpha=0.2)

In [135]:
# img generator config
img_gen_cfg = {
    "latent_features": latent_features,
    "latent_filters": latent_filters,
    "mask_value": mask_value,
    "return_sequences": True,
    "lstm_neurons": lstm_neurons,
    "latent_img_size": latent_img_size,
    "input_lyr_activation": ACT_REF,
    "latent_img_shape": latent_img_shape,
    "filters": filters, 
    "kernel_size": kernel_size,
    "stride": stride,
    "padding": padding,
    "hidden_lyr_activation": ACT_REF,
    "gen_dropout_rate": gen_dropout_rate,
    "output_filters": output_filters,
    "output_kernel_size": output_kernel_size,
    "output_stride": output_stride,
    "output_padding": padding,
    "output_shape": output_img_shape,
    "output_lyr_activation": output_gen_lyr_activation,
    }

print("GAN-img Generator Config:\n", img_gen_cfg)

GAN-img Generator Config:
 {'latent_features': 8, 'latent_filters': 1024, 'mask_value': 0.0, 'return_sequences': True, 'lstm_neurons': 400, 'latent_img_size': 65536, 'input_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002648616CDF0>, 'latent_img_shape': (8, 8, 1024), 'filters': 1024, 'kernel_size': (4, 4), 'stride': (2, 2), 'padding': 'same', 'hidden_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002648616CDF0>, 'gen_dropout_rate': 0.3, 'output_filters': 3, 'output_kernel_size': (3, 3), 'output_stride': (1, 1), 'output_padding': 'same', 'output_shape': (256, 256, 3), 'output_lyr_activation': 'tanh'}


In [136]:
# img discriminator config
img_dis_cfg = {
    "input_lyr_activation": ACT_REF,
    "input_filters": input_filters,
    "input_kernel_size": input_kernel_size,
    "input_stride": input_stride,
    "input_padding": input_padding,
    "filters": filters,
    "kernel_size": kernel_size,
    "stride": stride,
    "padding": padding,
    "hidden_lyr_activation": ACT_REF,
    "dis_dropout_rate": dis_dropout_rate,
    "mid_dis_neurons": mid_dis_neurons,
    "dense_cls_activation": ACT_REF,
    "output_dis_neurons": output_dis_neurons,
    "output_lyr_activation": output_dis_lyr_activation,
    "loss": LOSS_REF,
    "optimizer": DIS_OPTI_REF,
    "metrics": ACC_REF,
    }

print("GAN-img Discriminator Config:\n", img_dis_cfg)

GAN-img Discriminator Config:
 {'input_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002648616CDF0>, 'input_filters': 1024, 'input_kernel_size': (5, 5), 'input_stride': (2, 2), 'input_padding': 'same', 'filters': 1024, 'kernel_size': (4, 4), 'stride': (2, 2), 'padding': 'same', 'hidden_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002648616CDF0>, 'dis_dropout_rate': 0.2, 'mid_dis_neurons': 5000, 'dense_cls_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002648616CDF0>, 'output_dis_neurons': 1, 'output_lyr_activation': 'sigmoid', 'loss': 'binary_crossentropy', 'optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x000002648616CA00>, 'metrics': ['accuracy']}


In [137]:
# img GAN config
gan_cfg = {
    "loss": LOSS_REF,
    "optimizer": GEN_OPTI_REF,
    "metrics": ACC_REF,
    }

print("GAN-img Config:\n", gan_cfg)

GAN-img Config:
 {'loss': 'binary_crossentropy', 'optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x000002648616C970>, 'metrics': ['accuracy']}


### GAN-txt definition

In [138]:
# txt generator config
txt_gen_cfg = {
    "mask_value": mask_value,
    "input_return_sequences": input_return_sequences,
    "input_lstm_neurons": input_lstm_neurons,
    "input_lyr_activation": LeakyReLU(alpha=0.2),
    "mid_gen_neurons": mid_txt_gen_neurons,
    "lstm_neurons": lstm_neurons,
    "hidden_lyr_activation": LeakyReLU(alpha=0.2),
    "hidden_return_sequences": hidden_return_sequences,
    "gen_dropout_rate": gen_dropout_rate,
    "latent_lstm_reshape": latent_lstm_reshape,
    "memory_shape": memory_shape,
    "output_neurons": output_neurons,
    "output_shape": output_txt_shape,
    "output_lyr_activation": output_gen_lyr_activation,
    "output_return_sequences": output_return_sequences,
    }

print("GAN-txt Generator Config:\n", txt_gen_cfg)

GAN-txt Generator Config:
 {'mask_value': 0.0, 'input_return_sequences': True, 'input_lstm_neurons': 400, 'input_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x00000264872BB8B0>, 'mid_gen_neurons': 2130, 'lstm_neurons': 400, 'hidden_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x00000264872BB610>, 'hidden_return_sequences': True, 'gen_dropout_rate': 0.3, 'latent_lstm_reshape': (15, 142), 'memory_shape': (15, 142), 'output_neurons': 142, 'output_shape': (15, 142), 'output_lyr_activation': 'tanh', 'output_return_sequences': True}


In [139]:
# txt discriminator config
txt_dis_cfg = {
    "mask_value": mask_value,
    "input_return_sequences": input_return_sequences,
    "input_lstm_neurons": input_lstm_neurons,
    "input_lyr_activation": LeakyReLU(alpha=0.2),
    "lstm_neurons": lstm_neurons,
    "hidden_lyr_activation": LeakyReLU(alpha=0.2),
    "hidden_return_sequences": hidden_return_sequences,
    "hidden_lyr_activation": LeakyReLU(alpha=0.2),
    "memory_shape": memory_shape,
    "dis_dropout_rate": dis_dropout_rate,
    "mid_dis_neurons": mid_txt_gen_neurons,
    "dense_cls_activation": LeakyReLU(alpha=0.2),
    "output_dis_neurons": output_dis_neurons,
    "output_lyr_activation": output_dis_lyr_activation,
    "loss": "binary_crossentropy",
    "optimizer": Adam(learning_rate=0.000005, beta_1=0.5),
    "metrics": ["accuracy"],
    }

print("GAN-txt Discriminator Config:\n", txt_dis_cfg)

GAN-txt Discriminator Config:
 {'mask_value': 0.0, 'input_return_sequences': True, 'input_lstm_neurons': 400, 'input_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002645D540F70>, 'lstm_neurons': 400, 'hidden_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x00000264878C9D60>, 'hidden_return_sequences': True, 'memory_shape': (15, 142), 'dis_dropout_rate': 0.2, 'mid_dis_neurons': 2130, 'dense_cls_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x00000264872B8FA0>, 'output_dis_neurons': 1, 'output_lyr_activation': 'sigmoid', 'loss': 'binary_crossentropy', 'optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x0000026487864520>, 'metrics': ['accuracy']}


### CGAN-img definition

In [140]:
img_cgen_cfg = {
    "latent_features": latent_features,
    "latent_filters": latent_filters,
    "memory": memory_shape[0],
    "features": memory_shape[1],
    "mask_value": mask_value,
    "latent_img_size": latent_img_size,
    "input_lyr_activation": LeakyReLU(alpha=0.2),
    "latent_img_shape": latent_img_shape,
    "filters": filters, 
    "kernel_size": kernel_size,
    "stride": stride,
    "padding": padding,
    "hidden_lyr_activation": LeakyReLU(alpha=0.2),
    "gen_dropout_rate": gen_dropout_rate,
    "output_filters": output_filters,
    "output_kernel_size": output_kernel_size,
    "output_stride": output_stride,
    "output_padding": padding,
    "output_shape": output_img_shape,
    "output_lyr_activation": output_gen_lyr_activation,
    "labels_neurons": labels_neurons,
    "labels_lyr_activation": LeakyReLU(alpha=0.2),
    }

print("CGAN-img Generator Config:\n", img_cgen_cfg)

CGAN-img Generator Config:
 {'latent_features': 8, 'latent_filters': 1024, 'memory': 15, 'features': 142, 'mask_value': 0.0, 'latent_img_size': 65536, 'input_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x00000264861701C0>, 'latent_img_shape': (8, 8, 1024), 'filters': 1024, 'kernel_size': (4, 4), 'stride': (2, 2), 'padding': 'same', 'hidden_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x0000026486170A30>, 'gen_dropout_rate': 0.3, 'output_filters': 3, 'output_kernel_size': (3, 3), 'output_stride': (1, 1), 'output_padding': 'same', 'output_shape': (256, 256, 3), 'output_lyr_activation': 'tanh', 'labels_neurons': 2130, 'labels_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x0000026486170AF0>}


In [141]:
img_cdis_cfg = {
    "input_lyr_activation": LeakyReLU(alpha=0.2),
    "input_filters": input_filters,
    "input_kernel_size": input_kernel_size,
    "input_stride": input_stride,
    "input_padding": padding,
    "filters": filters,
    "kernel_size": kernel_size,
    "stride": stride,
    "padding": padding,
    "hidden_lyr_activation": LeakyReLU(alpha=0.2),
    "dis_dropout_rate": dis_dropout_rate,
    "mid_dis_neurons":mid_dis_neurons,
    "dense_cls_activation": LeakyReLU(alpha=0.2),
    "output_dis_neurons": output_dis_neurons,
    "output_lyr_activation": output_dis_lyr_activation,
    "labels_lyr_activation": LeakyReLU(alpha=0.2),
    "timesteps": memory_shape[0],
    "max_features": memory_shape[1],
    "labels_neurons": labels_img_neurons,
    "labels_lyr_activation": LeakyReLU(alpha=0.2),
    "labels_filters": labels_filters,
    "labels_kernel_size": labels_kernel_size,
    "labels_stride": labels_stride,
    "labels_reshape": labels_reshape,
    "loss": "binary_crossentropy",
    "optimizer": Adam(learning_rate=0.000005, beta_1=0.5),
    "metrics": ["accuracy"],
    }

print("CGAN-img Generator Config:\n", img_cdis_cfg)

CGAN-img Generator Config:
 {'input_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002648786B880>, 'input_filters': 1024, 'input_kernel_size': (5, 5), 'input_stride': (2, 2), 'input_padding': 'same', 'filters': 1024, 'kernel_size': (4, 4), 'stride': (2, 2), 'padding': 'same', 'hidden_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002648786BA90>, 'dis_dropout_rate': 0.2, 'mid_dis_neurons': 5000, 'dense_cls_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002648786B970>, 'output_dis_neurons': 1, 'output_lyr_activation': 'sigmoid', 'labels_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002648616CA30>, 'timesteps': 15, 'max_features': 142, 'labels_neurons': 65536, 'labels_filters': 1024, 'labels_kernel_size': (5, 5), 'labels_stride': (2, 2), 'labels_reshape': (50, 50, 3), 'loss': 'binary_crossentropy', 'opt

In [142]:
# txt GAN config
img_cgan_cfg = {
    "loss": "binary_crossentropy",
    "optimizer": Adam(learning_rate=0.00002, beta_1=0.5),
    "metrics": ["accuracy"],
    }

print("CGAN-img Config:\n", img_cgan_cfg)

CGAN-img Config:
 {'loss': 'binary_crossentropy', 'optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x000002645D59F7C0>, 'metrics': ['accuracy']}


### Multi CGAN definition (txt+img)

In [143]:
multi_cgen_cfg = dict()
multi_cgen_cfg.update(img_cgen_cfg)
multi_cgen_cfg.update(txt_gen_cfg)

print("Multi CGen-txt2img Config:\n", multi_cgen_cfg)

Multi CGen-txt2img Config:
 {'latent_features': 8, 'latent_filters': 1024, 'memory': 15, 'features': 142, 'mask_value': 0.0, 'latent_img_size': 65536, 'input_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x00000264872BB8B0>, 'latent_img_shape': (8, 8, 1024), 'filters': 1024, 'kernel_size': (4, 4), 'stride': (2, 2), 'padding': 'same', 'hidden_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x00000264872BB610>, 'gen_dropout_rate': 0.3, 'output_filters': 3, 'output_kernel_size': (3, 3), 'output_stride': (1, 1), 'output_padding': 'same', 'output_shape': (15, 142), 'output_lyr_activation': 'tanh', 'labels_neurons': 2130, 'labels_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x0000026486170AF0>, 'input_return_sequences': True, 'input_lstm_neurons': 400, 'mid_gen_neurons': 2130, 'lstm_neurons': 400, 'hidden_return_sequences': True, 'latent_lstm_reshape': (15, 142),

In [144]:
multi_cdis_cfg = dict()
multi_cdis_cfg.update(img_cdis_cfg)
multi_cdis_cfg.update(txt_dis_cfg)

mcdis_cfg_update = {
    "labels_lstm_neurons": labels_lstm_neurons,
    "labels_return_sequences": labels_return_sequences,
    "labels_img_reshape": labels_reshape,
    "labels_txt_reshape": labels_txt_reshape,
    "loss": "binary_crossentropy",
    "optimizer": Adam(learning_rate=0.000005, beta_1=0.5),
    # "optimizer": SGD(learning_rate=0.00020, momentum=0.3),
    "metrics": ["accuracy"],
    }

multi_cdis_cfg.update(mcdis_cfg_update)

print("Multi CDis-txt2img Config:\n", multi_cdis_cfg)

Multi CDis-txt2img Config:
 {'input_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002645D540F70>, 'input_filters': 1024, 'input_kernel_size': (5, 5), 'input_stride': (2, 2), 'input_padding': 'same', 'filters': 1024, 'kernel_size': (4, 4), 'stride': (2, 2), 'padding': 'same', 'hidden_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x00000264878C9D60>, 'dis_dropout_rate': 0.2, 'mid_dis_neurons': 2130, 'dense_cls_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x00000264872B8FA0>, 'output_dis_neurons': 1, 'output_lyr_activation': 'sigmoid', 'labels_lyr_activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000002648616CA30>, 'timesteps': 15, 'max_features': 142, 'labels_neurons': 65536, 'labels_filters': 1024, 'labels_kernel_size': (5, 5), 'labels_stride': (2, 2), 'labels_reshape': (50, 50, 3), 'loss': 'binary_crossentropy', 'opt

In [145]:
# txt2img CGAN config
multi_cgan_cfg = {
    "loss": "binary_crossentropy",
    "optimizer": Adam(learning_rate=0.0003, beta_1=0.5),
    "metrics": ["accuracy"],
    }

print("Multi CGAN-txt2img Config:\n", multi_cgan_cfg)

Multi CGAN-txt2img Config:
 {'loss': 'binary_crossentropy', 'optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x000002645D59FEE0>, 'metrics': ['accuracy']}


## ML Model Creation
### GAN img definition

In [146]:
# latent shape
latent_dims = 100
print(latent_dims)
# latent_shape = (int(X_img[0].shape[0]/4), int(X_img[0].shape[1]/4), 3)
# latent_shape = (100, 100)

100


In [147]:
gen_model = create_img_generator(latent_dims, img_gen_cfg)
print("GAN-img Generator Definition")
# dis_model = Sequential(slim_dis_layers)
gen_model.model_name = "GAN-img Generator"

# DONT compile model
# cdis_model.trainable = False
gen_model.summary()

GAN-img Generator Definition
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ImgGenIn (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
ImgGenDense_1 (Dense)        (None, 65536)             6619136   
_________________________________________________________________
ImgGenReshape_3 (Reshape)    (None, 8, 8, 1024)        0         
_________________________________________________________________
ImgGenConv2D_4 (Conv2DTransp (None, 16, 16, 1024)      16778240  
_________________________________________________________________
ImgGenBN_5 (BatchNormalizati (None, 16, 16, 1024)      4096      
_________________________________________________________________
ImgGenDrop_6 (Dropout)       (None, 16, 16, 1024)      0         
_________________________________________________________________
ImgGenConv2D_7 (Conv2DTransp (No

In [148]:
img_shape = X_img[0].shape
print(img_shape)

# img_shape = (100,100,3)
dis_model = create_img_discriminator(img_shape, img_dis_cfg)
print("GAN-img Discriminator Definition")
# dis_model = Sequential(slim_dis_layers)
dis_model.model_name = "GAN-img Discriminator"

# compile model
dis_model.compile(loss=img_dis_cfg["loss"], 
                    optimizer=img_dis_cfg["optimizer"], 
                    metrics=img_dis_cfg["metrics"])

# cdis_model.trainable = False
dis_model.summary()

(256, 256, 3)
GAN-img Discriminator Definition
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
DisImgIn (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
ImgDisConv2D_1 (Conv2D)      (None, 128, 128, 1024)    77824     
_________________________________________________________________
ImgDisConv2D_2 (Conv2D)      (None, 64, 64, 32)        524320    
_________________________________________________________________
ImgDisBN_3 (BatchNormalizati (None, 64, 64, 32)        128       
_________________________________________________________________
ImgDisDrop_4 (Dropout)       (None, 64, 64, 32)        0         
_________________________________________________________________
ImgDisConv2D_5 (Conv2D)      (None, 32, 32, 64)        32832     
_________________________________________________________________
ImgDisBN_6 (

In [149]:
print("GAN-img Model definition")
gan_model = create_img_gan(gen_model, dis_model, gan_cfg)
gan_model.model_name = "GAN-img"
gan_model.summary()

GAN-img Model definition
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ImgGenIn (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
ImgGenDense_1 (Dense)        (None, 65536)             6619136   
_________________________________________________________________
ImgGenReshape_3 (Reshape)    (None, 8, 8, 1024)        0         
_________________________________________________________________
ImgGenConv2D_4 (Conv2DTransp (None, 16, 16, 1024)      16778240  
_________________________________________________________________
ImgGenBN_5 (BatchNormalizati (None, 16, 16, 1024)      4096      
_________________________________________________________________
ImgGenDrop_6 (Dropout)       (None, 16, 16, 1024)      0         
_________________________________________________________________
ImgGenConv2D_7 (Conv2DTransp (None

In [150]:
# saving model topology into png files
print(timestamp)
export_model(gen_model, model_fn_path, gen_model.model_name, timestamp)
export_model(dis_model, model_fn_path, dis_model.model_name, timestamp)
export_model(gan_model, model_fn_path, gan_model.model_name, timestamp)

2021-06-12 11:26:33


### GAN txt definition

In [151]:
# gen_txt_model = create_txt_generator(latent_dims, txt_gen_cfg)
# print("GAN-txt Generator Definition")
# # dis_model = Sequential(slim_dis_layers)
# gen_txt_model.model_name = "GAN-txt Generator"

# # DONT compile model
# # cdis_model.trainable = False
# gen_txt_model.summary()

In [152]:
txt_shape = X_txt[0].shape
print(txt_shape)
# dis_txt_model = create_txt_discriminator(txt_shape, txt_dis_cfg)
# print("GAN-txt Discriminator Definition")
# # dis_model = Sequential(slim_dis_layers)
# dis_txt_model.model_name = "GAN-txt Discriminator"

# # compile model
# dis_txt_model.compile(loss=txt_dis_cfg["loss"], 
#                     optimizer=txt_dis_cfg["optimizer"], 
#                     metrics=txt_dis_cfg["metrics"])

# # cdis_model.trainable = False
# dis_txt_model.summary()

(15, 142)


In [153]:
print("GAN-txt Model definition")
# gan_txt_model = create_img_gan(gen_txt_model, dis_txt_model, gan_cfg)
# gan_txt_model.summary()
# gan_txt_model.model_name = "GAN-txt"

GAN-txt Model definition


In [154]:
# saving model topology into png files
print(timestamp)
# export_model(gen_txt_model, model_fn_path, gen_txt_model.model_name, timestamp)
# export_model(dis_txt_model, model_fn_path, dis_txt_model.model_name, timestamp)
# export_model(gan_txt_model, model_fn_path, gan_txt_model.model_name, timestamp)

2021-06-12 11:26:33


### CGAN definition

In [155]:
n_labels = y_labels[0].shape[0]
print(n_labels)
cgen_img_model = create_img_cgenerator(latent_dims, n_labels, img_cgen_cfg)
print("CGAN-img Generator Definition")
# dis_model = Sequential(slim_dis_layers)
cgen_img_model.model_name = "CGAN-img Generator"

# DONT compile model
# cdis_model.trainable = False
cgen_img_model.summary()

16


KeyboardInterrupt: 

In [ ]:
img_shape = X_img[0].shape
cdis_img_model = create_img_cdiscriminator(img_shape, n_labels, img_cdis_cfg)
print("CGAN-img Discriminator Definition")
# dis_model = Sequential(slim_dis_layers)
cdis_img_model.model_name = "CGAN-img Discriminator"

# compile model
cdis_img_model.compile(loss=img_cdis_cfg["loss"], 
                    optimizer=img_cdis_cfg["optimizer"], 
                    metrics=img_cdis_cfg["metrics"])

# cdis_model.trainable = False
cdis_img_model.summary()

In [ ]:
print("CGAN-img Model definition")
cgan_img_model = create_img_cgan(cgen_img_model, cdis_img_model, gan_cfg)
cgan_img_model.summary()
cgan_img_model.model_name = "CGAN-img"

In [ ]:
# saving model topology into png files
print(timestamp)
export_model(cgen_img_model, model_fn_path, cgen_img_model.model_name, timestamp)
export_model(cdis_img_model, model_fn_path, cdis_img_model.model_name, timestamp)
export_model(cgan_img_model, model_fn_path, cgan_img_model.model_name, timestamp)

### Multi CGAN-txt2img

In [ ]:
multi_cgen_model = create_multi_cgenerator(latent_dims, img_shape, txt_shape, n_labels, multi_cgen_cfg)
print("Multi CGAN-txt2img Generator Definition")
# dis_model = Sequential(slim_dis_layers)
multi_cgen_model.model_name = "Multi CGAN-txt2img Generator"

# DONT compile model
# cdis_model.trainable = False
multi_cgen_model.summary()

In [ ]:
multi_cdis_model = create_multi_cdiscriminator(img_shape, txt_shape, n_labels, multi_cdis_cfg)
print("Multi CGAN-txt2img Discriminator Definition")
# dis_model = Sequential(slim_dis_layers)
multi_cdis_model.model_name = "Multi CGAN-txt2img Discriminator"
# compile model

multi_cdis_model.compile(loss=multi_cdis_cfg["loss"], 
                    optimizer=multi_cdis_cfg["optimizer"], 
                    metrics=multi_cdis_cfg["metrics"])

# compile model
multi_cdis_model.summary()

In [ ]:
print("Multi CGAN-txt2img Model definition")
multi_cgan_model = create_multi_cgan(multi_cgen_model, multi_cdis_model, gan_cfg)
multi_cgan_model.summary()
multi_cgan_model.model_name = "Multi CGAN-txt2img"

In [ ]:
# saving model topology into png files
print(timestamp)
export_model(multi_cgen_model, model_fn_path, multi_cgen_model.model_name, timestamp)
export_model(multi_cdis_model, model_fn_path, multi_cdis_model.model_name, timestamp)
export_model(multi_cgan_model, model_fn_path, multi_cgan_model.model_name, timestamp)

In [156]:
print("-Images:", X_img.shape, "\n-Text:", X_txt.shape, "\n-Real/Fake:", y.shape, "\n-txt&img Labels:", y_labels.shape)

-Images: (59, 256, 256, 3) 
-Text: (59, 15, 142) 
-Real/Fake: (59, 1) 
-txt&img Labels: (59, 16)


In [157]:
def format_tfidf_tokens(tfidf_tokens):

    tfidf_dict = list()

    for tfidf in tfidf_tokens:

        tfidf = eval(tfidf)
        # print(type(tfidf), tfidf)
        td = dict(tfidf)
        tfidf_dict.append(td)

    return tfidf_dict

In [158]:
# training and batch size
gan_train_cfg = {
    "max_epochs": 500,
    "latent_dims": latent_dims,
    # "max_epochs": ini_config.get("Training", "MaxEpochs"),
    # "trained_epochs": ini_config.get("Training", "TrainedEpochs"),
    "batch_size": 32,
    "synth_batch": 1,
    "balance_batch": False,
    "gen_sample_size": 3,
    "models_fn_path": model_fn_path,
    "report_fn_path": report_fn_path,
    # "ini_fn_path": ini_fn_path,
    # "ini_cfg_fn": ini_fn,
    # "dis_model_name": multi_cgen_model.model_name,
    # "gen_model_name": multi_cdis_model.model_name,
    # "gan_model_name": multi_cgan_model.model_name,
    # "dis_model_name": cdis_img_model.model_name,
    # "gen_model_name": cgen_img_model.model_name,
    # "gan_model_name": cgan_img_model.model_name,
    "dis_model_name": dis_model.model_name,
    "gen_model_name": gen_model.model_name,
    "gan_model_name": gan_model.model_name,
    "check_epochs": 10,
    "save_epochs": 50,
    "max_save_models": 3,
    "latent_dims": latent_dims, # X_txt[0].shape,
    "pretrained": False,
    "conditioned": True,
    "dataset_size": X_img.shape[0],
    "img_shape": X_img[0].shape,
    "txt_shape": X_txt[0].shape,
    "label_shape": y_labels[0].shape,
    "cat_shape": y[0].shape,
    "data_cols": 2,
    # "data_cols": 3,
    # "data_cols": 4,
    "bow_lexicon": load_lexicon(lex_fn_path),
    "tfidf_lexicon": format_tfidf_tokens(tfidf_tokens.values),
    }

print("Model Training Config:\n", gan_train_cfg.keys())

Model Training Config:
 dict_keys(['max_epochs', 'latent_dims', 'batch_size', 'synth_batch', 'balance_batch', 'gen_sample_size', 'models_fn_path', 'report_fn_path', 'dis_model_name', 'gen_model_name', 'gan_model_name', 'check_epochs', 'save_epochs', 'max_save_models', 'pretrained', 'conditioned', 'dataset_size', 'img_shape', 'txt_shape', 'label_shape', 'cat_shape', 'data_cols', 'bow_lexicon', 'tfidf_lexicon'])


In [159]:
gan_data = (X_img, y)
# gan_data = (X_img, y_labels, y)
# gan_data = (X_img, X_txt, y_labels, y)
print(X_img.shape, X_txt.shape, y_labels.shape, y.shape)
print(len(gan_data))

(59, 256, 256, 3) (59, 15, 142) (59, 16) (59, 1)
2


In [390]:
# traininng with the traditional gan
training_model(gen_model, dis_model, gan_model, gan_data, gan_train_cfg)

# training with the conditional gan with images
# training_model(cgen_img_model, cdis_img_model, cgan_img_model, gan_data, gan_train_cfg)

# training with the muti conditional gan with images + text
# training_model(multi_cgen_model, multi_cdis_model, multi_cgan_model, gan_data, gan_train_cfg)

>>> Epoch: 1, B/Ep: 1/1, Batch S: 32 -> [R-Dis loss: 1.024, acc: 0.125] || [F-Dis loss: 1.014, acc: 0.062] || [Gen loss: 0.689, acc: 0.000]
Epoch: 1 Testing model training process...
No handles with labels found to put in legend.
1
Batch Size 32 -> Samples: Fake: 16 & Real: 16
>>> Test Fake -> Acc: 0.062 || Loss: 0.694
>>> Test Real -> Acc: 0.938 || Loss: 0.690
Ploting results
Epoch: 1 Saving the training progress...


OSError: Unable to create link (name already exists)

# THE END